In [3]:
from dotenv import load_dotenv
import os
import json

from openai import OpenAI


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
!pip install minsearch

import minsearch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [15]:
# get the FAQ  documents
with open('../01-intro/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [16]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [17]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [18]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

In [19]:
index.fit(documents)

In [20]:
open_ai_client = OpenAI()

In [21]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5 }
    
    results=index.search(
        query=query,
        # filter_dict={'course': 'data-engineering-zoomcamp'},
        filter_dict={'course': 'llm-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    context_template = """
    Q: {question}
    A: {text}
    """.strip()
    
    for response in search_results:
        context = context + context_template.format(question=response['question'], text=response['text']) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [23]:
def llm(prompt):
    response = open_ai_client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [24]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [25]:
query = "how do I run kafka"
answer = rag(query)
answer

"I'm sorry, but the current context from the FAQ database does not provide information on how to run Kafka. You might want to refer to Kafka's official documentation or relevant course materials for detailed instructions."

In [26]:
from elasticsearch import Elasticsearch

In [27]:
es_client = Elasticsearch("http://localhost:9200")

In [ ]:
index_settings={
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm

for document in documents:
    es_client.index(index=index_name, document=document)

In [ ]:
def elastic_search(query):
    
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
query = "How do copy a file to a Docker container?"

In [29]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
elastic_search_response = elastic_search(query)

context = ""
context_template = """
Q: {question}
A: {text}
""".strip()

for response in elastic_search_response:
    context = context + context_template.format(question=response['question'], text=response['text']) + "\n\n"

In [ ]:
prompt = build_prompt(query, elastic_search_response)

In [ ]:
print(prompt)

In [ ]:
len(prompt)

In [ ]:
!pip install tiktoken

In [ ]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
num_tokens = len(encoding.encode(prompt))
num_tokens

In [ ]:
#Bonus: generating the answer (ungraded)
answer = rag(query)
print(answer)

In [ ]:
#tokens in llm answer

num_tokens_anwer = len(encoding.encode(answer))
num_tokens_anwer

In [ ]:
#Bonus: calculating the costs (ungraded)
#$5.00 / 1M input tokens
#$20.00 / 1M output tokens

#Suppose that on average per request we send 150 tokens and receive back 250 tokens.
#How much will it cost to run 1000 requests?

def total_cost(requests, avg_input_tokens, avg_output_tokens, price_per_million_input_tokens, price_per_million_output_tokens):
    return requests * (price_per_million_input_tokens*avg_input_tokens/1e6 + price_per_million_output_tokens*avg_output_tokens/1e6)

estimated_cost = total_cost(1000, 150, 250, 5, 20)
estimated_cost

In [ ]:
# estimated cost from actual question and response:

# Q: -> prompt: 327 tokens
# Answer: answer: 91 tokens

In [ ]:
estimated_cost = total_cost(1, 327, 91, 5, 20)
estimated_cost

# RAG with vector search

In [30]:
from qdrant_client import QdrantClient, models

In [31]:
qd_client = QdrantClient("http://localhost:6333")

In [32]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [33]:
collection_name = "zoomcamp-faq"

In [35]:
# qd_client.delete_collection(collection_name=collection_name)

True

In [36]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [37]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [38]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [39]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.98it/s]


UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [40]:
question = 'I just discovered the course. Can I still join it?'

In [41]:
def vector_search(question):
    print('vector_search is used')
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [42]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

# Homework

In [64]:
#1 Q1. Embedding the query
collection_name='homework-collection'

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

question = 'I just discovered the course. Can I join now?'
vector = models.Document(text=question, model=model_handle)
point = models.PointStruct(
    id=1,
    vector=vector,
)

qd_client.upsert(
    collection_name=collection_name,
    points=[point]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [77]:
import numpy as np

In [78]:
question_vector = [-0.076394625,-0.07305555,0.058650177,0.039267056,-0.014179711,-0.046834286,0.026986282,0.034903906,0.0015341921,-0.011040736,0.05575563,-0.028836014,-0.04872395,-0.099092394,0.060257975,-0.018610537,-0.008810401,-0.030199483,-0.014254182,-0.038788423,-0.027612567,-0.006949575,0.032065332,-0.005178384,0.08379599,-0.08873296,-0.07302606,0.05928468,0.040780567,0.07726848,-0.04600297,0.03984269,0.01748215,0.008702637,-0.030929105,0.02204704,0.04764799,0.017656619,-0.03620136,-0.035340805,-0.00592538,0.02970912,0.08058838,0.010713272,-0.05381925,-0.009628811,-0.11726374,0.033654638,0.007057632,0.007701598,-0.051748253,0.054695558,-0.07802899,0.042180166,0.012625097,0.053133905,-0.026777718,0.01688207,-0.03750646,0.009131964,0.021369683,0.01601501,-0.050565016,0.0562206,-0.036680274,-0.038541663,-0.042125706,-0.01369216,-0.0858582,0.02626408,-0.061855566,-0.008855486,0.123171106,-0.067560956,-0.0043410063,0.015979148,-0.04543037,0.0761061,0.018493528,-0.030727323,-0.0437181,-0.0474577,-0.01959202,0.0123776775,-0.009961045,-0.06741564,0.025475208,0.012231816,0.055862892,-0.031431787,-0.0072399382,0.06332495,0.005724944,-0.06219999,-0.030861666,-0.042729013,0.05169637,0.032787953,-0.024481347,0.01791278,0.09482543,0.00021838388,0.070135616,-0.020138577,0.036499094,0.02130201,-0.05676774,-0.030814184,-0.113907054,-0.027006438,0.0026018482,-0.03290736,-0.025548322,-0.003436633,-0.05982757,0.016598605,0.072070286,-0.049084023,-0.04080078,-0.019638902,0.051647037,0.020605918,-0.019346183,0.037210956,0.053046416,0.07987352,0.00087196945,-0.0038696385,-0.010597066,0.0027043598,-0.02648081,0.024388878,-0.0296613,0.012427909,0.13307956,-0.034595076,-0.0045997603,0.03198323,-0.03035144,0.03814698,-0.04840417,-0.046379216,0.040090658,-0.048272986,-0.016706945,0.003551685,0.06379748,-0.0538861,-0.07283667,0.00010343121,-0.015575378,0.032012347,-0.06415197,-0.013540248,-0.0011958949,0.08672792,-0.0075801364,0.0523632,0.011338949,-0.029392216,-0.01965492,0.07334757,-0.018693488,-0.048615538,-0.043975666,-0.040023025,0.079713814,0.09002534,-0.051877894,0.008567889,0.057003487,0.021816235,-0.06553278,-0.010720462,-0.109933145,-0.03492299,-0.016764868,0.015357842,-0.041204426,-0.032461695,0.024459181,-0.038644724,-0.0274304,0.020797167,0.025932806,-0.044755038,-0.024058415,0.0073084934,0.019291757,-0.026945112,0.01387156,0.0069697835,0.011728565,-0.01833017,-0.02184551,0.002700352,-0.011002593,0.008522952,-0.0012190245,0.023168163,0.00807127,0.069403924,-0.059616663,0.035843354,-0.043555323,0.043354772,-0.041578364,-0.056675564,-0.054188706,-0.023901833,0.0056371484,0.023743743,0.07103455,0.017316848,-0.032258905,-0.019468198,-0.018814249,-0.023082504,0.033789493,-0.01827984,0.0038809841,0.018433804,-0.0045527765,0.026369426,-0.01903595,-0.026939003,0.0115375975,0.011661391,-0.008279113,-0.036442854,0.09963438,-0.029555894,-0.05389486,-0.025199503,0.012239184,-0.02270577,0.0051033124,-0.006740263,-0.02437843,0.00063222396,-0.072825775,0.031326637,0.06936266,-0.047581293,0.003878822,0.03424838,0.075902864,-0.009775129,0.04149426,0.073168136,-0.015485658,-0.011791858,-0.019198686,-0.0979174,-0.031492677,0.053558722,-0.020779178,0.06312393,-0.059724286,-0.023460502,0.016266406,0.053660795,-0.013769261,0.06598341,-0.009244944,-0.0019963982,-0.048756342,0.013951236,0.012251362,0.029866884,-0.008004716,-0.0059602675,0.028173184,-0.021334417,-0.03776228,-0.022715334,0.056092095,0.004302245,0.0363268,-0.03722529,-0.045394994,0.058543853,0.0151176425,0.015862223,0.07377338,-0.02138856,0.03215823,0.049821295,0.101769604,-0.012423644,0.0143238045,-0.09484546,-0.026360847,-0.0086600715,-0.009829075,0.07527368,0.03694464,0.03145399,0.0010480143,0.031180574,0.012896868,0.026071299,-0.047712754,-0.025566546,0.010813307,0.010325988,0.033903256,0.0027545742,-0.0414582,-0.018864492,-0.0077650216,-0.0258756,-0.015259804,0.03386696,0.06389977,0.039594736,0.06392791,-0.052033935,-0.060277577,0.01383048,-0.03389305,0.0082015665,0.042957243,0.027850358,0.046031557,0.031562913,-0.062610686,-0.020096386,-0.021295115,0.007169571,-0.021506857,0.028238242,-0.05885132,-0.09007525,-0.04328919,0.016374974,-0.064508975,-0.0718056,0.024650117,-0.0264174,0.038611546,0.012716956,0.0008036857,0.08491594,0.01611363,-0.004817931,0.020459484,0.016712865,-0.030300582,-0.012489704,-0.036335774,0.0014446833,-0.041627057,-0.014245733,-0.08835648,0.0062200367,-0.023921737,-0.07033747,-0.011145977,0.11117787,0.0068728556,-0.07548961,-0.017316684,0.062176384,0.042554203,-0.016942263,-0.016347172,-0.034440145,0.047769815,0.02331553,0.08948578,0.014258421,0.014397342,-0.058851037,0.032490604,-0.039942726,0.04315536,-0.0007629777,0.00014571798,-0.018599799,0.018144561,0.015716737,0.012985259,-0.056655545,-0.059225187,0.021767162,-0.017017864,0.023195872,0.05172877,0.0033785296,-0.043114323,-0.04113032,-0.03279466,0.021025563,0.046515483,-0.07140695,0.061237503,0.05745423,-0.017958986,0.07425888,-0.007829259,0.020093592,-0.045163617,-0.008470511,-0.040267102,-0.03998675,-0.077041894,-0.018193578,0.024872,0.010041384,0.05881555,0.016236473,0.0673376,0.013445556,-0.03371737,-0.07302301,-0.06799306,-0.027497325,-0.039959744,0.024465065,0.013153511,0.002455528,-0.036094945,0.027291806,0.062241435,0.0072020614,0.028018672,-0.031678937,0.05002582,0.030086052,0.0071202614,-0.03762181,-0.047790542,0.030331004,-0.070737176,0.046878066,0.023290593,-0.0825114,0.02578341,-0.008216676,0.1055644,-0.097606696,0.034995403,0.029698815,0.0605657,0.09999059,0.06002884,-0.027457938,-0.038310196,0.049868044,0.009857887,0.052792847,-0.047174033,0.022906117,-0.014179066,-0.0022146495,-0.0127322525,-0.100020014,0.021819761,0.027150447,0.03338376,-0.025846532,-0.034277286,-0.017014587,0.008481089,0.024005951,0.05044276,-0.0076720472,0.068071686,-0.017559806,-0.019640056,-0.08533978,-0.003290564,0.040699378,0.092369184,0.012957049,0.056126665,0.041953206,-0.058119684,0.09316495,-0.08035,-0.05468391,-0.06230982,-0.08776024,-0.032763798,-0.036852386,0.025452485,-0.0035177716,0.06201592,0.03891123,-0.04918638,-0.06718365,-0.012544886,-0.030297454,-0.04006706,0.042423878,0.024668565,-0.048333645,0.06881412,0.0417306,0.04731131,-0.076380335,0.01489839,0.04593115,-0.022789443,0.071536414,-0.020866815,-0.043364994,-0.07648079,0.0762217,0.03919631,0.012258728]
question_vector = np.array(question_vector)

In [79]:
min_value = min(question_vector)
min_value

np.float64(-0.11726374)

In [72]:
# Cosine similarity
doc = 'Can I still join the course after the start date?'
embedded_document = models.Document(text=doc, model=model_handle)
point = models.PointStruct(
    id=2,
    vector=embedded_document,
)

qd_client.upsert(
    collection_name=collection_name,
    points=[point]
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [80]:
document_vector = [-0.05453039,-0.07834517,0.031361036,0.023423493,-0.030632148,-0.055104163,0.022472648,0.015038616,-0.018115958,0.018059535,-0.0048089228,-0.07573959,-0.0048727216,-0.09077178,0.048394088,-0.04263174,-0.026146252,-0.044812914,-0.020692602,-0.037905447,-0.033032935,-0.008028502,0.011856035,-0.03837212,0.07263781,-0.058281176,-0.07308902,0.034091137,0.056070536,0.050268475,-0.059929334,0.05330166,-0.017952451,0.004177451,-0.02454279,0.013228825,0.046417832,0.010996376,-0.010149964,-0.0057752416,-0.00028398776,0.049149636,0.11121484,-0.027630642,-0.067892395,-0.018592827,-0.12396321,0.012574593,0.016529871,0.00722243,-0.027723745,0.051807355,-0.05986048,0.05273394,0.010384407,0.04820913,-0.022902327,0.015879998,-0.048051156,0.010248942,0.013788453,0.038744498,-0.057989437,0.05691391,-0.031520545,-0.026749857,-0.044805452,-0.040970754,-0.09204475,-0.005961161,-0.054933567,-0.0053422386,0.06680548,-0.05452177,-0.008698386,0.014044658,-0.039600894,0.039323676,-0.019675177,-0.0016311057,-0.00680347,-0.033159938,0.004858791,-0.038898908,-0.0034054453,-0.05198906,0.030815046,-0.027041137,0.06880273,-0.00085918495,-0.030557781,0.080685996,-0.018616946,-0.06453289,-0.05042765,-0.03781182,0.059976015,0.068733424,0.011212515,0.016454805,0.11303667,-0.014629214,0.10178224,-0.0340159,0.03438522,0.037586924,-0.05585729,0.0031079438,-0.116664074,-0.009978009,0.014995977,-0.068705074,-0.032911517,-0.005936865,-0.06926245,0.01508007,0.058511894,-0.05484877,-0.025717895,-0.035600748,0.00007275296,0.0005300028,-0.03352856,0.021270491,0.049506538,0.036636278,-0.0022505166,0.026512893,-0.032298975,-0.006862517,-0.013444221,-0.0080896355,-0.012910764,0.02480413,0.1401441,-0.03865883,0.0250565,0.038734935,-0.061451707,0.035439666,-0.042516425,-0.06416913,0.04632891,-0.032439746,-0.028825585,0.013096427,0.045363475,-0.021236088,-0.05389941,-0.0012804097,-0.020822262,0.039394222,-0.081791975,0.026144704,0.004937147,0.083967134,-0.028563328,0.046293434,0.013085121,0.00434111,-0.014611856,0.055602103,0.003255211,-0.06569591,-0.012480979,-0.009015182,0.04679276,0.071478486,-0.03473582,0.006190062,0.058234185,0.03136326,-0.08915606,0.023016494,-0.0731166,-0.024862193,-0.019507593,0.019011648,-0.038064577,-0.034119878,0.051055387,-0.03437948,-0.026698919,0.009701989,0.010824014,-0.07859339,-0.038456656,0.015683921,0.038044915,-0.022645421,-0.004786097,0.011510029,0.0036098398,-0.018999679,-0.017927181,0.0368781,-0.021001542,0.0236493,0.014056396,0.00352257,0.027135404,0.07652266,-0.060129084,0.030516246,-0.04588068,0.02248985,-0.02509205,-0.056375917,-0.022945173,-0.004278569,0.03571652,-0.0035525863,0.089150764,0.021267049,-0.0003747355,-0.022427155,-0.021966467,-0.041897092,0.026556825,-0.039378375,-0.0016830356,0.012646829,0.0044398494,0.05055788,-0.024474712,-0.039439254,0.015263813,0.012897372,-0.0031546513,-0.047259297,0.11782507,-0.052553657,-0.05324482,0.010515962,0.045807738,-0.02477396,-0.0025760366,-0.021975601,-0.038766768,-0.013610637,-0.049846414,0.0061211553,0.05663797,-0.032338437,0.0012003663,0.035274606,0.05160792,-0.036263138,0.056271072,0.053307362,-0.036344316,-0.040370345,-0.036199123,-0.06260612,-0.0083156815,0.041388202,-0.028873023,0.052249406,-0.07344021,-0.035356656,0.020984855,0.06310855,0.004609871,0.096370056,-0.009673757,-0.008952092,-0.07692271,-0.009608928,0.021477027,0.02300094,0.0031054178,0.005051526,0.01639365,-0.0206681,-0.060601786,-0.030388305,0.049644213,0.020335022,0.03663503,-0.039529584,-0.051106025,0.0316693,0.004380434,0.022118751,0.0562609,-0.014186501,0.03596959,0.075952105,0.10604726,0.0014501987,0.0060323565,-0.09005736,-0.0050126743,-0.032307144,-0.0024683527,0.086979955,0.050612364,0.02369656,0.008115339,0.055449378,0.017799936,0.031119984,-0.035562243,-0.025292479,0.023145527,0.015994104,0.03101153,-0.015452211,-0.016324352,-0.062596366,0.0057359883,-0.046848092,-0.024846582,0.042961046,0.053224888,0.0017727263,0.033780564,-0.08398649,-0.053805046,0.01189494,-0.0066188597,-0.038960204,0.057747617,0.020715637,0.035991173,0.05058722,-0.064457856,0.0033200516,0.011072353,-0.018658182,-0.018573835,0.0012730731,-0.04628667,-0.07171918,-0.041946135,-0.0013151848,-0.029280607,-0.08032365,0.012932714,-0.03370907,0.029310388,-0.0037580275,-0.0014484517,0.10054827,-0.01005649,-0.024626609,0.04053921,0.0033588628,-0.053446963,-0.02543281,-0.015254357,-0.01437095,-0.018567711,-0.0021396894,-0.087300174,-0.016063368,0.030246165,-0.034906294,-0.022564948,0.100616135,0.0030022236,-0.055097762,-0.023553915,0.09525224,0.069201574,-0.021848504,0.00024909907,-0.034907065,0.04737357,0.03601434,0.08327148,0.04125495,0.025414348,-0.048555493,0.027400425,-0.03804268,0.059127808,0.025324497,-0.03369107,-0.019090163,0.029626172,0.024338102,-0.02391626,-0.045128852,-0.027256839,0.010933056,0.018631082,-0.001849716,0.009468776,-0.01155594,-0.024924396,-0.035072453,-0.04037387,0.04196305,0.06676912,-0.073026225,0.07563604,0.060634624,-0.001560672,0.02906267,-0.019871293,0.021563092,-0.03380162,-0.007296022,-0.024438905,-0.016348017,-0.07874989,-0.025695456,0.014030669,0.0032540893,0.05299744,0.018384442,0.034186568,0.03955367,-0.038830318,-0.07369271,-0.090413995,-0.047558777,-0.026291909,0.008760703,0.018520445,0.0057121143,-0.023133755,0.03007515,0.0558853,0.006046486,0.04203799,-0.044314835,0.060901634,0.021554597,-0.015515915,-0.051812723,-0.028861178,0.049642075,-0.017378176,0.022264957,0.018409314,-0.05657597,0.052607987,-0.0043727285,0.097556844,-0.087618254,0.038492337,0.034306936,0.100228235,0.08323109,0.013972967,-0.02920884,-0.026461279,0.04505955,0.045076825,0.06562655,-0.025714861,0.043814104,-0.009092007,0.032456685,-0.010052404,-0.12289983,0.026378334,0.04243696,0.041651703,-0.056828886,-0.0915505,0.01930859,-0.011346786,-0.012421942,0.025725473,0.033811547,0.01955349,-0.009046328,0.0048167906,-0.06696646,0.0010412439,0.021235198,0.09026971,0.03460806,0.05179775,0.044768326,-0.06940265,0.09753953,-0.046439618,-0.060506783,-0.07177515,-0.061659854,-0.042283703,0.0006492992,0.014800269,-0.0043354295,0.099197425,0.023680244,-0.05637402,-0.06303828,0.0064743957,-0.02774474,-0.04935212,0.00895176,0.00634117,-0.03777013,0.084666856,0.03988926,0.04745732,-0.07418304,0.040468942,0.065727144,-0.0106213745,0.029607808,-0.006053229,-0.03269042,-0.07250637,0.07731074,0.0561754,0.0062051597]
document_vector = np.array(document_vector)

In [81]:
question_vector.dot(document_vector)

np.float64(0.9008528945283214)

In [82]:
# Ranking by cosine
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [83]:
collection_name = 'ranking_by_cosine'

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

points = []

for i, doc in enumerate(documents):
    text = doc['question']# + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [85]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [86]:
doc_embeddings = np.array([
np.array([-0.0579807,-0.08410192,0.030685335,0.017524691,-0.03272382,-0.054813746,0.027203402,0.019029917,-0.01651686,0.016074039,-0.007507367,-0.08306511,-0.013441365,-0.09132376,0.04860769,-0.041941855,-0.026747521,-0.04144771,-0.025556458,-0.038844086,-0.033573687,-0.008521531,0.015283255,-0.041125633,0.07460193,-0.057825573,-0.066362984,0.03328208,0.06022811,0.04700542,-0.053268068,0.056874998,-0.017195206,0.003745213,-0.02567105,0.011895528,0.052484427,0.011941201,-0.0067951037,-0.014727434,-0.0067965486,0.043101717,0.109834366,-0.023189709,-0.06794427,-0.020430932,-0.11977992,0.014443698,0.011668445,0.009521466,-0.02479742,0.045216784,-0.058254804,0.050771136,0.011179785,0.049738996,-0.021041807,0.013401556,-0.057245716,0.006396251,0.0061334665,0.037901923,-0.057744768,0.054961998,-0.03645428,-0.026743777,-0.047612343,-0.030699894,-0.09001697,-0.0051594824,-0.057897136,-0.0074732266,0.07066468,-0.057670698,-0.00606619,0.01822544,-0.04592727,0.04379796,-0.015358282,-0.00043695947,-0.015079044,-0.035494674,0.0036891918,-0.03794775,0.0057732365,-0.054574415,0.028513506,-0.026129862,0.06904142,-0.0032089443,-0.02341362,0.07723747,-0.011993993,-0.06268998,-0.050707914,-0.03777783,0.064775765,0.067098916,0.004058606,0.011784543,0.10736673,-0.016675964,0.098054014,-0.030544989,0.023895567,0.036519628,-0.054699678,0.0059876423,-0.11827173,-0.011401749,0.016748909,-0.07033202,-0.03713521,-0.0046923663,-0.06643819,0.01846197,0.05979334,-0.05172434,-0.025883274,-0.035114296,0.0039326316,0.001595003,-0.024500724,0.01770846,0.043521833,0.039031785,-0.0005698675,0.024729384,-0.0325474,-0.011879659,-0.011211416,-0.009359453,-0.01909404,0.02598124,0.13692899,-0.039797843,0.020400936,0.039999265,-0.059319343,0.037135776,-0.050274454,-0.065229125,0.0516172,-0.038831856,-0.028173337,0.016416771,0.044564225,-0.014895981,-0.058865257,0.006478621,-0.02323091,0.036490463,-0.08217508,0.018121542,0.0053413804,0.07923856,-0.02660238,0.047387872,0.016805349,0.0011768077,-0.0072400942,0.053812847,0.0055986387,-0.055048853,-0.0143411765,-0.01671609,0.04597714,0.07572846,-0.036717154,0.006865197,0.0556024,0.029460583,-0.08803189,0.021207908,-0.07772848,-0.024701204,-0.021828135,0.021113865,-0.036868155,-0.032286208,0.046344966,-0.029811228,-0.025863858,0.006018752,0.015340381,-0.07212361,-0.037844867,0.024250543,0.04273116,-0.018597366,0.0013891908,0.011393058,-0.0015725411,-0.020352304,-0.009902864,0.03739213,-0.03046627,0.022891488,0.015165356,-0.0011232874,0.031864647,0.07290902,-0.06695748,0.029242607,-0.04711298,0.015573246,-0.027241565,-0.05756455,-0.026425593,-0.002355292,0.030770995,-0.0056457273,0.09452907,0.018664869,-0.001706808,-0.02626015,-0.024925234,-0.04277772,0.031971585,-0.03480128,-0.002793065,0.01693314,0.004422811,0.046651773,-0.021028604,-0.03829456,0.014194021,0.017780902,-0.0033105863,-0.04548633,0.12067077,-0.055606227,-0.046435382,0.011783658,0.043428086,-0.021862507,-0.00090971973,-0.016683865,-0.033591233,-0.006300096,-0.04863833,0.007878672,0.056753796,-0.035448365,0.00087361556,0.03573107,0.055642527,-0.0380202,0.061034445,0.05727691,-0.035083897,-0.04228019,-0.037291083,-0.07036947,-0.008829836,0.047180943,-0.024633238,0.05348281,-0.07018198,-0.039533395,0.01676696,0.06564256,0.011238847,0.09015853,-0.0075921193,-0.00817242,-0.075670436,-0.006861071,0.025415754,0.0195247,0.011545715,0.004347129,0.012890856,-0.01640336,-0.05594426,-0.027109215,0.049075205,0.009025141,0.044771805,-0.038703363,-0.05116064,0.03268455,0.0032897468,0.02383516,0.0639339,-0.016704788,0.032779235,0.079038575,0.10398852,-0.0060956534,0.0070750825,-0.08664575,-0.0034544119,-0.029804742,-0.0035957969,0.085739106,0.054635085,0.020617167,0.0020189076,0.055012494,0.01923742,0.029637102,-0.035733547,-0.023326544,0.020252082,0.011523631,0.027836034,-0.014333714,-0.01458893,-0.060090415,0.009479467,-0.04152081,-0.027012164,0.038273156,0.055029895,0.0031240564,0.034362644,-0.08212589,-0.04821212,0.014601511,-0.013927645,-0.033227075,0.060732238,0.02645,0.037242364,0.046158995,-0.06617073,0.0014898881,0.008882214,-0.02434447,-0.019094754,0.0046747075,-0.047134526,-0.07132881,-0.037217204,0.0019694539,-0.024854446,-0.08163205,0.014884341,-0.02743764,0.02546292,0.001506862,0.0069522336,0.10684125,-0.008623267,-0.022592712,0.041719552,0.0067786244,-0.053562142,-0.028984046,-0.019468337,-0.014763667,-0.019383432,0.0011834976,-0.08931652,-0.0103064105,0.018230999,-0.043116692,-0.021080315,0.10366546,0.0025039306,-0.05544197,-0.016394744,0.09426057,0.07182456,-0.027400065,0.0004924806,-0.039453372,0.04666407,0.040084105,0.08698682,0.043326054,0.030392647,-0.04931627,0.03029327,-0.03250252,0.06014299,0.025418995,-0.034350026,-0.023318699,0.022767654,0.023118684,-0.0137037905,-0.04595221,-0.026033843,0.016947396,0.014218642,-0.0013567752,0.017533112,-0.017756125,-0.027202787,-0.03689803,-0.037696246,0.04454887,0.06525985,-0.07947843,0.07027743,0.05422039,0.00009822509,0.030158488,-0.020298487,0.021676393,-0.032765646,-0.0043018865,-0.024223013,-0.013322752,-0.07832853,-0.017888792,0.017202206,0.006325235,0.050099883,0.021977153,0.038759522,0.040517937,-0.035617474,-0.07145943,-0.090277284,-0.04976991,-0.02623007,0.01486819,0.013535567,0.008678228,-0.028835109,0.036022548,0.057912424,0.0061541023,0.042966444,-0.041822545,0.061013803,0.024525724,-0.008424576,-0.049285587,-0.027669925,0.047040045,-0.027415397,0.022833671,0.015504734,-0.06024441,0.054680936,-0.010521723,0.09486545,-0.083075516,0.03566012,0.035076067,0.09929219,0.08165425,0.013946996,-0.034135357,-0.030354224,0.046986397,0.042603426,0.06257494,-0.024334645,0.03723336,-0.007890589,0.03377546,-0.011200295,-0.12407239,0.029675826,0.039797906,0.03126869,-0.05625994,-0.08606323,0.014141865,-0.016364016,-0.008107658,0.03178677,0.03446245,0.021863831,-0.013143512,0.0024515868,-0.08094097,-0.009315697,0.024083046,0.08271276,0.02851035,0.05236805,0.04479312,-0.061061602,0.102204226,-0.049311392,-0.067702256,-0.06995422,-0.055957746,-0.040529236,-0.0024166072,0.021367317,-0.0015608877,0.094978906,0.02286087,-0.058293857,-0.056961477,0.004701196,-0.034693796,-0.056149006,0.010747446,0.0107926,-0.042131163,0.07772144,0.046595182,0.046814837,-0.0831812,0.036436647,0.06601414,-0.0063816123,0.031255454,-0.010832911,-0.032461625,-0.0737731,0.08134946,0.061550304,0.0025028894]),
np.array([-0.08788074,-0.07376762,0.023896493,0.016774952,-0.042957757,-0.04790129,0.02843708,0.0426919,-0.007748879,0.013875758,0.021217898,-0.082565725,-0.04160864,-0.087023415,0.041464634,-0.039816037,-0.010003016,-0.016916929,-0.0185551,-0.032510154,-0.053226862,-0.004297053,0.05286379,-0.039630838,0.0750042,-0.065328546,-0.044588305,0.04262386,0.055458385,0.082857035,-0.046217293,0.06974232,-0.021933248,0.0064975508,-0.02231495,-0.009521015,0.06904913,0.02594995,-0.00033724136,-0.043723714,-0.030422209,0.016022928,0.12594031,0.00692337,-0.07237019,0.024875754,-0.11136151,0.017710274,0.02682748,0.017670799,-0.0040810737,0.028747378,-0.03700947,0.07681742,0.03161607,0.03062735,-0.019184,-0.020454127,-0.062361535,0.021376505,-0.0076697487,0.04608311,-0.059880447,0.039876603,-0.015990268,-0.034312516,-0.0082358895,-0.015602406,-0.107910566,0.021836068,-0.07166438,-0.00034633742,0.07410417,-0.06692731,0.013135788,0.035627797,-0.04769312,0.07026879,-0.020866118,-0.014528154,-0.0329965,-0.024583444,-0.009874887,-0.0023430188,0.0029525147,-0.055457063,0.02458217,0.016239079,0.06540336,-0.02713157,-0.0017835365,0.06226386,0.0019117297,-0.06921732,-0.028968245,-0.030341953,0.048516605,0.038964555,0.007168235,0.027854202,0.08674615,-0.019435449,0.085727796,-0.029380219,-0.0056931623,0.05699113,-0.064507686,-0.010235362,-0.11199941,-0.007897833,-0.0011566398,-0.06636121,-0.03026067,0.011849326,-0.03281752,0.018334271,0.050862588,-0.044095267,-0.011369201,-0.027356328,0.02541282,-0.018874101,-0.01960189,0.032999158,0.027262144,0.045491654,-0.012396475,0.031090343,-0.04797252,-0.009614835,-0.0071011013,-0.032743447,0.0026128292,0.006682852,0.11277128,-0.030647973,0.015036859,0.04211579,-0.058762792,0.03394914,-0.062478654,-0.06782684,0.05004565,-0.044189792,-0.011792724,0.021612506,0.042644765,-0.006700001,-0.07271722,0.001974218,-0.031378753,0.020435266,-0.091798946,0.013691679,0.006653586,0.07387118,-0.029445589,0.03570724,0.026196482,0.00090019405,0.005501828,0.03310649,0.006448854,-0.03854057,-0.0057025654,-0.036942206,0.097414605,0.08230706,-0.05468691,-0.012944635,0.06159875,0.0068488074,-0.061732247,-0.026017748,-0.072176605,0.00035306255,-0.049276985,0.02033875,-0.02060707,-0.037890192,0.036228925,-0.033971727,-0.017796753,0.019046847,0.013964533,-0.04626795,0.007064121,0.023091419,0.030984152,-0.021859998,-0.0072091967,-0.0032684014,-0.019811437,-0.012489453,0.013869885,0.03890519,-0.037096187,0.027863622,0.013264856,-0.008820358,0.028282424,0.058392,-0.05269317,0.03976642,-0.06592025,0.017718693,-0.022190683,-0.04233572,-0.042878408,0.0069674007,0.03945011,-0.037254058,0.039868474,-0.011770049,-0.038501363,-0.008289305,-0.023053428,-0.023672046,0.04197558,-0.04596362,-0.00089945237,0.029695995,-0.018068744,0.05334664,-0.011344861,-0.035218224,0.009503759,0.02818475,0.00446189,-0.044237647,0.09720891,-0.06731486,-0.05906997,0.01660403,0.045560606,-0.007856465,-0.009849266,-0.009800315,-0.017046558,-0.0019518142,-0.03261875,0.057139326,0.056341633,-0.048230533,-0.007266268,0.06190419,0.047737584,-0.054301675,0.044410173,0.06601646,-0.009649625,-0.022453353,-0.050939526,-0.11314754,-0.016255876,0.07655598,-0.032177214,0.055197522,-0.040830906,-0.039489545,0.013581153,0.055390146,0.01127232,0.06099352,-0.0032703774,0.0041820104,-0.07297633,-0.027204536,0.04019042,0.01604834,0.05263218,0.004042296,0.009792133,-0.021966849,-0.06475427,-0.015173524,0.059702817,0.019979298,0.03224392,-0.03432419,-0.04272997,0.042397276,-0.016678764,-0.0040921844,0.072722115,-0.008042753,0.034249317,0.08196315,0.089863904,-0.006185244,0.02650306,-0.08029345,-0.015753524,-0.02178348,0.0053206654,0.082846485,0.06602057,0.018190594,0.008673291,0.040011276,0.023522355,0.022774234,-0.03969117,-0.047113877,0.057636414,-0.007846029,0.015231703,-0.007952909,-0.016267722,-0.037586883,0.0005454707,-0.018504972,-0.02565413,0.006802607,0.040936645,-0.0060473587,0.074634165,-0.057720035,-0.017856425,-0.00063820346,-0.06877129,-0.024800183,0.05682034,0.04115262,0.027256703,0.029887943,-0.055466693,-0.011674414,-0.00048380357,-0.0052357977,-0.022835294,-0.008171454,-0.068433404,-0.08868935,-0.012379609,0.010056565,-0.03199862,-0.10063865,-0.007934568,-0.02059697,0.007769567,-0.003534364,-0.01619769,0.11832499,0.0003054544,-0.019536221,0.014578719,0.028200284,-0.060515907,-0.048952322,0.011354095,-0.009847516,-0.054223478,-0.008068937,-0.093978494,-0.0149174575,-0.021101987,-0.033068188,0.0025652503,0.105623275,-0.0030713256,-0.068122946,-0.008294549,0.06938329,0.04518291,-0.005683006,0.005127019,-0.025774362,0.031968992,0.054110426,0.07052933,0.010260684,0.054704282,-0.030915394,0.028821565,-0.047957607,0.051196583,-0.019263238,-0.038236815,-0.03983261,0.02006484,0.056913197,0.01196156,-0.061835296,-0.013390479,0.039193016,-0.022604732,-0.001984674,0.022552356,-0.008969588,-0.033802256,-0.042082787,-0.034871917,0.05021928,0.062383473,-0.065047234,0.06171641,0.07036622,0.010335998,0.011147329,-0.01400933,0.030603645,0.014904706,0.017557036,-0.019033512,-0.024825003,-0.06470651,-0.006933633,0.022806061,0.015428195,0.049497504,0.017021816,0.03913425,0.037636753,-0.03799051,-0.06630806,-0.057080146,-0.027691977,0.0044190795,0.03846963,0.003406529,0.0009317845,-0.02378163,0.023794102,0.04981688,0.023500094,0.031032775,-0.017145438,0.037852876,0.017536586,0.014642773,-0.067447305,-0.024861269,0.04476717,-0.05463799,-0.0044015734,-0.018465338,-0.0882292,0.06795156,-0.011285989,0.091971144,-0.08944417,0.028283458,0.02434575,0.06595732,0.05819654,0.057290126,-0.048858218,-0.04409184,0.048755746,0.04555155,0.04255841,-0.036989104,0.031401113,-0.022508478,-0.014228566,-0.0074781897,-0.11840047,0.0061246,0.014810875,0.026778372,-0.030074144,-0.073691055,0.057703454,-0.0050385036,-0.001119943,0.049536455,0.010305978,0.005665541,-0.014360273,-0.02703328,-0.10003487,-0.028189687,0.03989517,0.0721914,0.00079018937,0.06432768,0.05290469,-0.04574588,0.108626865,-0.06488934,-0.07741123,-0.083711706,-0.06950584,-0.035057884,-0.022422815,0.021217704,0.02951917,0.09131059,0.034629453,-0.04691602,-0.057994064,-0.009027554,-0.053730283,-0.042034723,0.0066553312,0.024792744,-0.046383526,0.040269837,0.05462087,0.04538637,-0.092012994,0.033626594,0.059899617,0.012651231,0.051971465,-0.0121005075,-0.024056695,-0.059542287,0.09366396,0.051859953,0.0071122237]),
np.array([-0.089363575,-0.0877183,0.037913308,0.027994243,0.00595548,-0.047237173,0.0141375875,0.044277262,0.000037387636,0.019119473,0.038240798,-0.05324963,-0.016459791,-0.10074006,0.071064204,-0.039555628,-0.013612759,-0.033777848,0.004635788,-0.03164673,-0.015497532,-0.01909324,0.03182538,-0.065793514,0.08282089,-0.080294594,-0.0823035,0.023911035,0.04823324,0.09224978,-0.038041856,0.06822365,-0.018035466,0.027590595,-0.03387082,-0.032378662,0.047537815,0.019917801,-0.0243114,-0.06535747,-0.026155153,0.013173191,0.101517044,0.027336344,-0.06378002,0.016160589,-0.13003151,0.02662851,0.026024966,0.01796365,-0.021471666,0.033547007,-0.060655806,0.044615403,0.009705083,0.04708202,-0.011652011,0.002331738,-0.045388903,-0.0034965668,0.009791054,0.024742771,-0.06687338,0.07016819,-0.015916897,-0.01928825,-0.021028554,-0.03517801,-0.089308456,0.007872618,-0.048782963,-0.009389237,0.04356737,-0.058133226,0.019508623,-0.019465862,-0.040022407,0.07099635,-0.032028053,-0.023945997,-0.015394815,-0.039365336,-0.022941256,0.0011170392,0.005754157,-0.06624569,-0.0017423445,-0.034388855,0.040933628,-0.029275898,-0.0014594767,0.03082589,0.00076076057,-0.068058796,-0.044848744,-0.018067509,0.010156723,0.04508392,-0.015969569,0.028900698,0.09362414,-0.022599375,0.102880664,-0.048520945,0.028181648,0.026041148,-0.07559772,-0.021057043,-0.09739337,-0.03458507,0.019411488,-0.07723491,-0.048153505,0.0068242205,-0.034477457,0.00012782798,0.040164407,-0.036251552,-0.021775883,-0.006020385,0.02596063,-0.019717606,-0.044731658,0.02759718,0.011602954,0.064634286,0.012339697,0.02112371,-0.03213414,0.001480462,-0.0017265807,0.0029046077,0.01213164,0.010199558,0.14238186,-0.057208303,0.024662914,0.012957289,-0.04967768,0.04220186,-0.029170973,-0.04428363,0.04049844,-0.080883734,-0.014044678,0.04547708,0.050674494,-0.02276494,-0.07273188,0.031217258,-0.060685705,0.019158201,-0.09251155,-0.031345487,-0.017957442,0.078574434,-0.036092192,0.06465936,0.01359916,0.02300308,-0.014829242,0.050600693,-0.015525642,-0.01630357,-0.01798351,-0.04959237,0.08302858,0.08788341,-0.035275064,0.0028598402,0.06126225,0.0165187,-0.09053564,0.014616655,-0.07536539,-0.016007604,-0.026337676,0.011258774,-0.0640473,-0.04254552,0.03489729,-0.056428477,-0.008771722,0.03539994,0.0072158976,0.0012314742,-0.019432103,0.005835443,0.042848546,-0.019290388,-0.005096332,0.015448293,0.0028772296,-0.02252912,-0.009512756,0.01643633,-0.018561946,0.03717108,0.01808583,0.02114724,0.019161567,0.07005297,-0.083690695,0.027541945,-0.056481253,0.0059611904,-0.029869717,-0.03916932,-0.029855276,0.0028317983,0.021231068,-0.010831128,0.07302275,0.039349258,0.0009229557,-0.0024339827,-0.03297946,-0.025455315,0.048264064,-0.020908404,-0.0026091875,0.0037351064,-0.003309638,0.061647654,-0.0033996752,-0.021484368,0.00053509674,0.01660646,0.005882735,-0.030459434,0.0947891,-0.044560578,-0.07052849,-0.01109853,0.025115103,-0.029753752,-0.017742256,-0.03260639,-0.033924866,0.011163469,-0.0012480917,0.03776726,0.03275477,-0.039923206,0.025166428,0.06914403,0.09419751,-0.030610561,0.040265262,0.043453597,-0.035214692,0.0018061011,-0.030739777,-0.09892229,-0.05684854,0.03239299,-0.013546637,0.06401812,-0.03804424,-0.024784438,0.004572098,0.03587366,0.02153894,0.05823616,-0.008687399,-0.03812003,-0.05963359,-0.024397794,0.048107803,0.007028115,0.0006356755,0.009091965,0.005221512,-0.020675713,-0.053224362,0.003333401,0.07153925,0.031286746,0.05919673,-0.04271645,-0.05256928,0.025568934,0.0010767621,-0.03151756,0.07407514,-0.012209481,0.045164756,0.09068019,0.11281911,0.0066970247,0.013710438,-0.06849526,0.006148426,-0.021065053,0.0010183455,0.06241948,0.041496124,0.03194189,-0.036312077,0.05478889,0.028179355,0.028811721,-0.026402975,-0.033893675,0.01192364,-0.025911659,0.020703373,-0.003228387,-0.010615817,-0.031688903,0.013944381,-0.007005586,-0.064662084,0.005549948,0.05516873,0.0056051495,0.05837894,-0.048734616,-0.035745893,-0.022757351,-0.019979427,-0.049888927,0.049224596,0.0127571225,0.056286085,0.06750418,-0.033062246,-0.016772747,0.0021148664,0.007058369,-0.04613114,0.001355367,-0.047146007,-0.07664062,-0.015147662,0.004988563,-0.038167972,-0.080477186,0.025259743,-0.066407524,0.04120361,0.01845707,-0.0069381613,0.078634396,0.006645078,-0.020704085,0.03909765,0.005366345,-0.05502412,-0.020835154,0.009860625,-0.013958296,-0.05673204,-0.0027480908,-0.088462695,-0.0049418136,-0.029059414,-0.05266473,-0.0027149513,0.09034685,0.001556634,-0.09470187,-0.02428045,0.083823666,0.060069166,-0.024253733,0.01058098,-0.03417508,0.030419366,0.052982707,0.048842404,0.044399664,0.031650938,-0.05353615,0.0382184,-0.038339745,0.06438003,-0.045287028,-0.025707852,-0.030485561,0.02587661,0.058392975,0.0053412444,-0.08864692,-0.03313087,0.012051994,-0.0013878212,0.039782327,0.04148843,-0.014301895,0.0033502264,-0.01789199,-0.024679564,0.019812336,0.014079525,-0.09983817,0.052529603,0.052656338,0.008204339,0.051864147,-0.008087819,0.024518218,-0.014157231,-0.0016653126,-0.023897426,-0.012073216,-0.05559991,0.0044258926,0.04267252,0.037498474,0.048522457,0.011540564,0.018455539,0.0336388,-0.031150537,-0.08756115,-0.090016,-0.008872893,0.0136869345,0.011087807,0.01637576,0.0041043023,-0.02113269,0.015199373,0.03597956,0.011465745,0.029487496,-0.02399838,0.048843145,0.0027937694,0.0059501505,-0.05528087,0.010665755,0.046355497,-0.048198618,0.04627908,0.0016474585,-0.03754697,0.072668985,-0.027749345,0.10700157,-0.070436895,0.031498265,0.046078153,0.046718597,0.05167602,0.050148666,-0.0036444655,-0.052289296,0.022262862,0.081773266,0.0384007,-0.033924703,0.037709404,-0.018126268,-0.029467536,-0.018170103,-0.10007221,0.034617744,0.028448788,0.015582866,-0.010123622,-0.048202053,0.029025786,-0.0069423467,-0.014976579,0.086020745,0.0118088955,0.026933739,0.0028631263,-0.039057735,-0.07537666,-0.019073697,0.0581317,0.07525456,0.011298148,0.032289945,0.021049164,-0.018019198,0.06733622,-0.07323523,-0.10245263,-0.060979452,-0.09242817,-0.04226574,-0.0060092895,0.03553691,-0.0054633347,0.0916176,0.021137126,-0.07208052,-0.036065955,0.026183203,-0.009932352,-0.063664064,0.008881581,0.03127808,-0.031415284,0.05091485,0.04342426,0.04725626,-0.07566957,0.015282173,0.029736282,-0.009417783,0.043376748,-0.030740084,-0.033508655,-0.0924483,0.08952052,0.077884436,-0.016288597]),
np.array([-0.10051171,-0.092732355,0.0032590884,0.025613287,-0.025349613,-0.06409159,0.013444444,0.02271292,-0.010152332,0.039222576,-0.013464915,-0.057785608,-0.0626387,-0.10794934,0.06530537,-0.04259154,-0.004588984,-0.019977735,-0.016333302,-0.06398826,-0.006703166,-0.035063937,0.034916885,-0.05663482,0.05237101,-0.05782926,-0.07311207,0.00086951203,0.052543666,0.08011161,-0.052982133,0.06333031,-0.0271786,0.016013892,-0.055240676,-0.030840063,0.07531441,0.018496823,-0.028338801,-0.044393536,-0.01823742,0.014187797,0.14053608,0.00677487,-0.04859617,-0.013796263,-0.10420956,0.033126872,0.0029681246,0.01969268,-0.030678347,0.0447267,-0.044761214,0.06071839,0.017049138,0.05803582,0.009363704,0.017575052,-0.025545646,-0.0059463284,0.021686222,0.00012521728,-0.050441436,0.0861865,-0.02382654,-0.01010181,-0.033319976,0.0018377068,-0.070775785,0.00889902,-0.07668494,-0.0021674982,0.05921154,-0.066805094,0.03617405,-0.0015465029,-0.029643686,0.06288182,-0.014974828,-0.038680356,-0.06154939,-0.044966403,-0.027200393,0.006353267,0.026934886,-0.05677624,0.008171769,-0.01439172,0.076927,-0.023670472,-0.013428825,0.0099274395,0.012318443,-0.06890801,-0.057280328,-0.029710082,0.07579128,0.042954735,0.027052293,0.031344544,0.104207635,0.00425021,0.09037418,-0.025543004,0.017579444,0.012299529,-0.067377776,-0.03165591,-0.08814341,-0.031107595,0.01263694,-0.050362173,-0.04326018,0.021212544,-0.038748585,-0.0043690177,0.06393138,-0.052346185,-0.024468254,-0.025668524,0.019822283,-0.004165701,0.00016555274,0.054706078,0.01509431,0.041247062,-0.0343081,0.023094755,-0.041107036,0.008205951,-0.021031298,-0.01595353,0.02046563,0.06112087,0.08567059,-0.06672479,0.009932042,0.002425325,-0.05785336,0.02826983,-0.019917622,-0.027758703,0.070067294,-0.039775144,-0.019347874,0.007294451,0.07945189,-0.007942574,-0.0682812,0.038611576,-0.03399065,0.029467585,-0.054763965,-0.0047920165,0.005631868,0.0852434,-0.0396236,0.022242915,0.032975264,-0.0045438586,-0.0055788467,0.024418386,0.008038419,-0.024933008,-0.035531666,-0.04897086,0.06423981,0.11017015,-0.022386966,-0.02390119,0.07400421,-0.0002799937,-0.06198506,-0.017616296,-0.05623236,-0.035271507,-0.029695245,0.02231283,-0.062801056,-0.05691907,0.032401443,-0.041170616,-0.020414371,0.00427475,0.0034016022,-0.013435382,-0.013744514,0.02444933,0.027257694,-0.06093615,-0.04477741,0.007536391,0.008282111,-0.03165465,-0.02196546,0.03248312,-0.054533303,0.027824203,0.020836817,0.030292252,-0.0016799599,0.07018231,-0.054361314,0.0034177895,-0.06942121,-0.012679468,-0.022801472,-0.04871996,-0.040190414,0.020060854,0.0163956,-0.015043655,0.075258106,-0.006535835,-0.008155594,0.020359844,0.0063227,-0.010253042,0.022858255,-0.04456446,-0.0100922445,-0.00006395343,0.0009672373,0.046199203,-0.019092247,-0.024577616,0.046924625,-0.0019186856,-0.014451766,-0.044407718,0.07342523,-0.053989846,-0.05096359,0.0011577215,0.031171028,-0.030000543,-0.013178018,-0.030874629,-0.04602516,0.0073461113,-0.018998448,0.04470323,0.048563823,-0.042167787,0.013613839,0.054699168,0.08703639,-0.048381653,0.027934644,0.05996542,-0.029498177,-0.005717091,-0.036634006,-0.084929496,-0.04095029,0.05286774,0.013703935,0.08555148,-0.02690152,-0.027443614,0.0152453175,0.03911221,-0.022845684,0.06957807,0.035677955,0.011218084,-0.08433594,-0.031320885,0.052316997,0.0029231904,-0.028139064,-0.005425975,0.007001101,-0.05908126,-0.07831384,0.012910393,0.03480763,0.0020537206,0.044599157,-0.030183652,-0.06697938,0.03788415,-0.0074483086,0.0074067325,0.02935319,-0.000017915034,0.04775491,0.060849775,0.08836668,-0.008662309,0.0012503408,-0.073644616,-0.0024781337,-0.02201468,-0.0020878029,0.052265998,0.06218688,0.02786867,-0.008179408,0.011425015,0.03872296,0.04748981,-0.038559318,-0.0263198,0.019614836,-0.0059438217,0.012533752,-0.01566317,-0.031062426,-0.045674622,-0.004933285,-0.028559176,-0.039798018,-0.015346421,0.060152024,-0.012297947,0.0496792,-0.019731492,-0.004724081,0.005631293,-0.036800303,-0.011590254,0.06992635,0.041173138,0.034148756,0.09278321,-0.03418118,-0.02327704,-0.006998322,-0.022490066,-0.039908424,-0.012456061,-0.06200842,-0.073398255,-0.007001286,0.004982901,-0.017678628,-0.088136956,-0.010103956,-0.03411945,0.04796718,0.02254102,-0.02451922,0.0977238,0.021888215,-0.026282316,0.012179013,0.0024762254,-0.053940713,-0.024719918,-0.00054912094,-0.024545116,-0.04133927,-0.017571464,-0.100050785,-0.01697244,-0.005674953,-0.045435198,-0.013118005,0.08643931,-0.014206349,-0.10268238,-0.013988042,0.089116305,0.039723992,-0.015354352,0.012000578,-0.029869048,0.02037352,0.047598023,0.04301618,0.028113823,0.04914273,-0.043981213,0.013573301,-0.03743431,0.04894342,-0.02581623,-0.01651894,-0.03198242,-0.009670053,0.048107553,0.008009532,-0.08042637,-0.026590625,-0.004267243,0.022947285,0.024848526,0.03745199,-0.0017173695,0.004667833,-0.05059137,-0.03475746,0.06299798,0.046471454,-0.07880094,0.06013078,0.057791732,0.03426622,0.024795953,0.0035455641,0.031940214,-0.025279544,0.00347023,-0.031380214,-0.028424492,-0.06451064,-0.010328864,0.019905077,0.03711139,0.057505004,0.018079618,0.03570284,0.050754633,-0.009285791,-0.047986597,-0.075482026,-0.008141463,0.01023723,0.010744678,0.015319578,0.017867813,-0.03776204,0.040294517,0.04309619,0.03140618,0.004927352,-0.021393502,0.06746736,-0.0020090858,0.021416117,-0.059847765,0.0027218622,0.08385108,-0.033527374,0.039190907,0.031758483,-0.054377686,0.046537332,-0.02350863,0.12419758,-0.08899877,0.03295906,0.05231114,0.09671444,0.048952643,0.05542874,-0.039278593,-0.039344463,0.036658663,0.07157355,0.045912366,-0.005410919,0.054540638,-0.02333255,-0.015023507,-0.030668905,-0.06406811,0.0021066517,0.011516566,0.015715411,-0.015833694,-0.08563288,0.028585596,-0.031803522,-0.0009192598,0.08479477,-0.016495798,0.042383652,0.008093974,-0.029900862,-0.075664915,0.0129625015,0.05506529,0.08313874,0.019638076,0.046725117,0.007392593,-0.03895062,0.058637347,-0.047840435,-0.06505317,-0.092574425,-0.11595671,-0.03123624,-0.0012528066,0.013190083,-0.0235686,0.07866425,0.046116043,-0.060054712,0.0052474057,0.0149595775,0.006940995,-0.047305685,0.018024765,0.048746623,-0.046216942,0.070958,0.029279076,0.04446096,-0.056868974,0.054578424,0.06481848,0.00051752565,0.052368306,-0.037384454,-0.011402074,-0.09055542,0.06255039,0.08245938,-0.009379261]),
np.array([-0.092314035,-0.056814525,0.023957947,0.041772105,0.0017493507,-0.044567622,0.03011164,0.039257515,-0.018937837,-0.009443904,0.03618455,-0.03296605,-0.029079027,-0.098473765,0.06242098,-0.029146742,-0.002283475,0.0053358735,-0.026081778,-0.055571087,-0.01879969,-0.038191598,0.050962787,-0.0065275934,0.080897786,-0.041530937,-0.0715715,0.04621445,0.04649537,0.06596111,-0.04052829,0.06671002,-0.047608428,0.026968531,-0.07416277,0.01256992,0.042464007,0.031869125,-0.020940347,-0.07380444,-0.05993911,0.028210804,0.10095978,0.012208029,-0.06944931,-0.002125375,-0.12142844,0.03098276,0.0064883647,0.023085747,-0.037092455,0.06989612,-0.068354115,0.06870955,0.013426214,0.04720347,-0.0009079065,0.031912774,-0.024926376,0.008126203,0.030990513,0.0069293934,-0.035527214,0.08974178,-0.038752537,-0.044814534,-0.038068946,-0.013076408,-0.05863419,0.05861668,-0.06471576,-0.033548355,0.05718545,-0.072146565,0.016720103,0.011880184,-0.039739583,0.078363776,0.029568044,-0.044394698,-0.035565853,0.00033732923,-0.0102887815,-0.0070530726,0.016045202,-0.047944807,0.009114408,0.05161541,0.071184665,-0.030922526,-0.0041058105,0.006246513,-0.024859311,-0.0616895,-0.012361393,-0.018345874,0.08119995,0.027213901,0.011763144,0.04363521,0.09465247,0.0067570936,0.079174064,-0.026551556,0.060774397,0.027012307,-0.06702549,-0.024873177,-0.10585515,-0.029578157,-0.011812733,0.000678551,-0.011459703,0.02983959,-0.044254944,0.002142705,0.031027766,-0.04268414,-0.043658275,-0.0040374594,0.04880442,0.022349738,-0.029384606,0.046402227,0.028842384,0.070283584,0.0013790794,0.024866382,-0.02665102,0.02870028,-0.07459509,-0.007327012,-0.006352046,0.03482161,0.113600515,-0.07135289,0.048440993,0.033318724,-0.040910274,0.0421193,-0.06691426,-0.0063592526,0.04652027,-0.07652258,-0.03153741,-0.015045254,0.062181804,-0.03506167,-0.08861173,0.049792808,0.004748015,-0.017822057,-0.09842575,-0.0194861,0.014891532,0.09395182,-0.05922519,0.06296735,0.05372674,-0.015771432,-0.01483436,0.03433638,-0.00966539,-0.039693117,-0.009049049,-0.03330967,0.06748453,0.088331535,-0.0481071,-0.015228475,0.09764069,0.0025773626,-0.086820915,-0.018281026,-0.071162924,-0.024173709,-0.007994019,0.034257893,-0.008607534,-0.051844176,0.03532546,-0.03757602,-0.017739464,-0.0035545102,0.025028959,-0.055305872,-0.006016125,0.0045478554,0.013047096,-0.0004293049,-0.042126134,0.008445642,0.03479457,-0.013234737,-0.028872877,0.041398846,-0.006122736,0.04796119,0.0027877695,0.03400188,0.02165671,0.057792217,-0.035277307,0.035485636,-0.05976896,-0.001215954,-0.00080356497,-0.06415254,-0.016203292,0.015882382,0.025767526,0.00995299,0.04009834,0.016862746,0.003522215,-0.0061938427,0.018604828,-0.042221528,0.037908714,-0.013524586,0.016699994,0.0092814155,-0.022646936,0.026340498,-0.031429876,-0.025634104,0.006463224,-0.007508525,0.016652182,-0.044821948,0.025716258,-0.04651361,-0.08704733,-0.006245379,0.032579742,-0.029381653,-0.0040500867,-0.02583833,-0.016956715,-0.01667298,-0.034311436,0.045585867,0.07789048,-0.04194058,0.019719837,0.06369598,0.106456086,-0.03951548,0.05658261,0.027510053,0.01913977,-0.023104457,-0.030307177,-0.09527151,-0.05640874,0.05482811,0.022817308,0.060886275,-0.031537153,-0.05148145,0.017156467,0.02124825,-0.03495524,0.07905644,0.011254447,-0.03015236,-0.046529062,-0.0060507134,0.043210547,0.017426664,0.024733894,-0.014035436,0.0009139147,-0.050636344,-0.08211056,-0.025055086,0.033253413,0.020484712,0.04881012,-0.04292943,-0.07400544,-0.019519698,-0.039535534,-0.0009096043,0.05607537,-0.030111607,0.040566143,0.045583192,0.06010472,0.010864437,0.006421008,-0.09444406,0.00039365233,0.008868011,-0.02873986,0.07674926,0.036318906,0.0029076603,-0.029241784,0.023907373,0.045304213,0.033942256,-0.034802105,-0.007839685,0.034801763,-0.006274953,0.025658617,0.013154079,-0.036259696,-0.028226662,0.0059041725,-0.05921824,-0.029226018,-0.016640743,0.06637955,0.02667369,0.08169221,-0.03259928,-0.011477818,-0.006677474,-0.08765335,-0.015170259,0.046010308,0.02505079,0.036457453,0.061332975,-0.028030844,-0.043243058,-0.027065858,-0.011579376,-0.07097634,0.000622023,-0.034270257,-0.07091576,-0.0060826447,-0.010531993,-0.04257324,-0.09628908,-0.019800495,-0.05980346,0.05228976,0.030014755,-0.034923296,0.09156322,0.013942526,0.0025476962,0.01113862,-0.024853898,-0.049989674,-0.021797635,-0.022418264,-0.01974025,-0.0382957,0.0047318907,-0.10726486,-0.014285749,-0.03256223,-0.046504676,-0.035571508,0.11057786,0.0018207309,-0.083738364,-0.033191696,0.08858694,0.015324059,-0.023428775,-0.009732042,-0.016580185,0.05532601,0.023302127,0.061096523,0.017796943,0.031361766,-0.051831085,0.043168075,-0.018291805,0.05448673,-0.022829482,0.0069835223,-0.04232058,-0.021066446,0.062498715,0.008481692,-0.062051367,-0.008425646,0.030062765,0.029347103,0.032736346,0.013876977,0.00048786117,-0.011949976,-0.055850815,-0.018149469,0.048869647,0.037204273,-0.07158382,0.016039446,0.08591553,-0.021004321,0.023250194,0.0002087554,0.04401624,-0.026634404,-0.005245466,-0.015542373,-0.021786818,-0.051745795,-0.012979426,0.03782313,0.008737757,0.08452764,0.009226153,0.03723291,0.017037462,-0.008708633,-0.077925436,-0.063981295,0.0021876574,0.0009624483,0.043758996,0.039439734,0.027728366,-0.00070335745,0.02816157,0.029660305,0.038672317,0.0126150735,-0.0034745412,0.068590574,0.016368335,0.010768403,-0.05052806,-0.03064047,0.015126323,-0.03986105,0.032587733,0.03582199,-0.05402454,0.0604012,0.0119586,0.1132202,-0.11162515,0.019863088,0.028117388,0.062161963,0.06546131,0.0386358,-0.016401108,-0.016268436,0.03815962,0.049520124,0.046032246,-0.015090378,0.02966519,-0.0465802,0.0070964466,-0.02492702,-0.08407265,0.031311568,0.028352836,0.023279237,-0.024696806,-0.044715255,0.010977562,-0.0011627885,0.0054377383,0.058460157,0.0005675663,0.033560514,0.011770569,-0.032481767,-0.06855121,0.013587438,0.0282629,0.04911807,0.013573371,0.016245853,0.027638225,-0.06988367,0.05749279,-0.062334128,-0.042833548,-0.09156335,-0.13281436,-0.04769229,-0.007587999,0.018569523,0.0023197965,0.042072527,0.03452595,-0.03999472,-0.046966575,-0.017443417,-0.026851838,-0.049966436,0.031224422,0.049497556,-0.054551944,0.07237708,-0.02160543,0.04542175,-0.07194036,0.008770576,0.032825585,-0.0077639003,0.039106973,-0.027971184,-0.054875202,-0.07597627,0.029711807,0.018294651,0.00591898]),
])

In [91]:
similarity = doc_embeddings.dot(question_vector)
similarity

array([0.90870672, 0.88460185, 0.87782715, 0.85116093, 0.86534703])

In [92]:
#Ranking by cosine, version two

collection_name = 'ranking_by_cosine_v2'

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [93]:
doc_embeddings_v2 = np.array([
np.array([-0.039587483,-0.069040686,0.01146618,0.026260398,-0.024734102,-0.024720706,0.003439712,0.042785924,-0.024162939,-0.026199237,-0.014921451,-0.07881784,-0.024740089,-0.07740145,0.07686427,-0.046568375,-0.030845447,-0.044565722,-0.044814143,-0.060655635,-0.036711324,-0.01800094,0.031952273,-0.06805734,0.059340086,-0.0710709,-0.08380713,0.07134354,0.07235555,0.02434782,-0.055102468,0.029459167,-0.018308243,0.025609976,-0.03172869,0.027675109,0.049635854,0.013883946,-0.020100303,-0.0028251265,-0.02937191,0.028263273,0.10064843,0.0028943666,-0.07165612,0.0007678534,-0.08024244,0.011240784,0.025550382,0.015388172,-0.03343803,0.02859503,-0.07989595,0.034472,-0.007653963,0.06632288,-0.010571334,0.024312265,-0.040355776,-0.0028037445,-0.001748615,0.03015687,-0.053803626,0.055337235,-0.010549608,-0.036919646,-0.030680675,-0.02739266,-0.10284297,0.016560767,-0.09235248,0.019417351,0.07478364,-0.037187453,-0.0028790287,0.042985857,-0.036176495,0.063270204,0.0069727395,-0.03654856,-0.0071746972,0.014704002,0.0049941316,-0.040363748,0.037977125,-0.03524048,0.05373871,-0.009404613,0.06290837,0.0031993077,-0.018874079,0.058727406,-0.022026582,-0.056836657,-0.03839625,-0.05403721,0.054665122,0.06276627,0.012520438,0.015159201,0.06398781,-0.0009202561,0.09304316,-0.019144364,0.01152079,0.046101432,-0.018310137,-0.013496304,-0.111972794,-0.0025632484,0.0025320528,-0.039448198,-0.015993172,-0.014560081,-0.055432536,-0.0068669026,0.06268585,-0.07159822,-0.04360402,-0.04990684,0.02317062,-0.0035471339,-0.022134097,0.035740163,0.04639596,0.05812364,-0.015089627,0.029095212,-0.050909698,0.021188995,-0.013351502,0.0015092266,-0.05344747,0.030104414,0.12214827,-0.052581407,0.039252054,0.04417843,-0.07737741,0.060089007,-0.052560363,-0.056129076,0.04935619,-0.034109488,-0.007529476,-0.006210679,0.04265892,-0.006359576,-0.048412994,0.023889245,-0.01704969,0.045763746,-0.09533837,0.025613725,-0.00022342024,0.08146816,-0.025151223,0.05422165,0.026569903,-0.008363569,-0.020065937,0.06510898,0.007590835,-0.059963144,0.0030241825,0.010344357,0.033102587,0.09050836,-0.04638574,0.00032408442,0.06650034,0.03184558,-0.07797178,0.0053156167,-0.07789726,-0.051195648,-0.043922666,0.01853223,-0.05596939,-0.027670676,0.02069573,-0.033162232,-0.015070117,0.014604954,0.007797157,-0.07986222,-0.03684022,0.002593958,0.053145222,-0.00008088464,-0.0067121675,-0.006367047,-0.014548667,-0.030095134,-0.023606764,0.062427007,-0.069108725,0.056857627,0.010098312,0.017539421,0.010375565,0.061139423,-0.026033754,0.040747516,-0.047514282,0.03284775,-0.007963655,-0.04709741,-0.040508848,0.0012606152,0.019879643,0.0025686303,0.09233436,-0.0054400493,-0.00768134,-0.05042883,0.015020456,-0.048081312,0.010918954,-0.018509556,0.010897814,0.018964913,0.004691618,0.053250335,-0.012111927,-0.05660649,0.018095355,0.00014703092,-0.0024964488,-0.02877134,0.07940119,-0.07872933,-0.046614926,0.034798983,0.04167991,-0.03781998,-0.0025943487,-0.033810176,-0.06620857,-0.025126588,-0.034623828,0.021199608,0.086035356,-0.016463645,-0.0015300402,0.04249392,0.036412768,-0.05379808,0.062544495,0.053096335,-0.030777674,-0.05144378,-0.054193053,-0.07130077,-0.0032815402,0.036987845,-0.0011352513,0.067471504,-0.06891754,-0.033971846,0.0063266447,0.067035094,0.012146904,0.07942248,0.0033933623,0.0014343574,-0.080087125,0.0009883242,0.034039218,-0.0015514793,0.01605497,0.0072586853,0.0019016908,-0.020441057,-0.048919346,-0.044031665,0.080284745,0.023977801,0.05823703,-0.028308244,-0.04357746,0.013442698,0.011008718,0.007877841,0.047267705,-0.01572112,0.00027324836,0.058208756,0.10677266,-0.014757419,0.027855692,-0.103960544,0.015092182,-0.024054872,0.008098654,0.08798257,0.030487765,0.02166618,0.012615643,0.017521607,0.008373162,0.015300104,-0.037111014,-0.038546722,-0.006021168,0.0038125264,0.07523739,-0.0059082983,-0.023376854,-0.03904999,0.009422153,-0.022484694,0.006228613,0.046920497,0.07741533,0.026500328,0.03900372,-0.09576224,-0.069260634,0.01843344,-0.014474392,-0.042309426,0.062137555,0.04289044,0.015587401,0.07927791,-0.07659772,0.016719949,-0.0046637016,-0.054756895,-0.033748165,-0.023334691,-0.049304005,-0.059880503,-0.039799206,-0.0023161557,0.002792598,-0.05759132,0.047322977,-0.03416652,0.035849854,-0.013568037,0.002905318,0.104250915,-0.010750141,-0.031868983,0.022189481,-0.001591123,-0.084124506,-0.0021448545,-0.0010949944,-0.011057108,-0.0113222925,0.0049857763,-0.067887336,0.0041005234,0.0021352174,-0.0203391,-0.025997099,0.09947086,0.0070350366,-0.03930016,-0.013743858,0.066257276,0.07938004,-0.040847734,-0.01427264,-0.038295384,0.04181416,0.05209792,0.066787995,0.023743192,0.034430943,-0.04759543,0.009740686,-0.01370084,0.061390452,-0.002682962,-0.021099167,-0.025313584,0.0064885044,0.047148444,-0.034992736,-0.03909267,-0.027514301,0.013291716,0.033204738,0.001808036,-0.006416668,-0.017876538,-0.037507348,-0.01764928,-0.032299574,0.04546062,0.07446243,-0.09018406,0.0780735,0.052254405,-0.021331517,0.027134364,-0.00008821763,0.016917793,-0.021865008,-0.02097325,-0.014949604,-0.038147315,-0.0679762,-0.010931504,0.035468966,0.014643073,0.059162218,0.014112825,0.0025679283,0.06897744,-0.0038894026,-0.037046574,-0.06572652,-0.042674948,-0.031436406,0.016323246,0.03353908,0.019246694,-0.049653776,0.0440218,0.030164026,0.011753882,0.041184634,-0.036086235,0.065380335,0.03333631,0.008967064,-0.053544503,-0.029791564,0.034911614,-0.021235846,0.01587038,-0.0056805075,-0.066986896,0.046339795,-0.016466733,0.07108736,-0.07521951,-0.0036566576,0.030737612,0.12080991,0.07019251,-0.008805508,-0.031624608,-0.00724157,0.013749234,0.065587446,0.0707025,-0.025254546,0.042926773,-0.017257279,0.031012189,-0.013259278,-0.10905859,0.04187871,0.045645744,0.050856385,-0.027635409,-0.061516006,0.015847297,-0.0064714383,0.00007785194,0.017610736,0.034634694,0.027170323,-0.005895106,0.00083315506,-0.084535874,-0.010413376,0.048145417,0.08003395,0.057008013,0.048376203,0.03525333,-0.0582114,0.06650339,-0.03320102,-0.07946924,-0.075734,-0.05951169,-0.057963215,-0.000430385,0.0245066,0.011386469,0.08302691,0.04182244,-0.06247924,-0.026919235,-0.026191147,-0.044487994,-0.049721863,0.033994198,0.032355875,-0.036453974,0.06565758,0.026142443,0.049676046,-0.05656706,0.032462597,0.03825758,0.011823763,0.011324448,-0.0069513423,-0.037014708,-0.06353737,0.091140315,0.047398962,-0.010331829]),
np.array([-0.08249755,-0.08627812,0.005936041,0.01717951,-0.035829112,-0.042589147,0.014288602,0.050919767,-0.023475086,-0.009658801,0.01513574,-0.06999786,-0.0124821905,-0.09058999,0.038457736,-0.03783834,-0.0056239464,-0.02563267,-0.043781158,-0.037560686,-0.05816893,-0.0022800537,0.06694992,-0.024348157,0.09433825,-0.06986983,-0.05221443,0.07069592,0.055142682,0.049884968,-0.038953558,0.072943695,-0.0155285755,0.0042785797,-0.029693201,-0.012692787,0.04826572,0.020786919,-0.027600788,-0.060529612,-0.032036945,0.020629443,0.10627858,0.041157305,-0.04761458,0.020037737,-0.09621812,0.03505008,0.022286346,0.025970878,-0.03411613,0.022206074,-0.042295687,0.0695523,0.009151945,0.04968523,-0.009599207,-0.00034746918,-0.056706704,0.028889379,0.012326159,0.019487105,-0.073899895,0.037583563,-0.007938229,-0.052890923,-0.03070966,-0.021579329,-0.11566398,0.026709218,-0.0873283,-0.001076729,0.1046458,-0.06367595,0.015311773,0.048514858,-0.03976653,0.074582405,0.005372081,-0.03152442,-0.045927335,0.017992219,0.001771981,0.018517295,0.030959764,-0.054081455,0.033921942,0.012874104,0.04196766,-0.037883177,0.01130129,0.029545626,-0.007157468,-0.059071142,-0.0071576503,-0.05123697,0.052822057,0.030040104,0.018879844,0.023878908,0.07381184,-0.022831352,0.08899873,-0.017865682,0.033885583,0.07689245,-0.049345236,0.008847437,-0.107514635,-0.020615438,0.015390262,-0.036083728,-0.041866016,0.009472945,-0.024835136,0.014642163,0.026585339,-0.061080214,-0.03952787,-0.04767755,0.015136055,-0.0029060931,-0.017173484,0.049031444,0.039665665,0.084811024,0.00064251764,0.057234485,-0.07891437,0.0026135556,0.0036441148,-0.019183883,0.005613923,0.004146622,0.09775148,-0.028535042,0.044844963,0.03976793,-0.059765942,0.03570411,-0.075066514,-0.043056816,0.01981507,-0.042365365,0.017938659,0.019705866,0.037038073,0.008473041,-0.07456053,0.0064021526,-0.028204767,0.008605629,-0.07045589,0.017036213,-0.0024761118,0.08215069,-0.040213775,0.02835133,0.030089723,0.0012857346,0.0010201706,0.048206747,-0.0071060476,-0.040682115,-0.0034164714,0.008740739,0.07265202,0.08803254,-0.061051875,-0.0072979955,0.08194728,-0.01759995,-0.07590446,-0.039662223,-0.07386134,0.005959266,-0.05723852,0.05532186,-0.03517832,-0.024162272,0.029763319,-0.049503434,-0.010873067,0.018910794,0.023660246,-0.047474924,0.016991382,0.032871895,0.042317983,-0.0030024503,-0.00042078015,0.0031422826,-0.02150263,-0.016046215,0.00051924575,0.06628081,-0.049698852,0.036813516,-0.009482053,0.0035860303,0.04468988,0.04050218,-0.036225457,0.028292807,-0.06427288,0.022875128,-0.019176358,-0.034112822,-0.0203982,0.012276142,0.032515462,-0.018409055,0.025129732,-0.0024323876,-0.028875973,-0.015979229,-0.023865571,-0.028035108,0.0037199436,-0.025196245,0.006143104,0.036959067,-0.031007973,0.03757419,-0.02294356,-0.04948265,0.008699835,-0.0016772766,-0.0039747665,-0.02910382,0.067931354,-0.08735695,-0.055851348,0.04953612,0.04602587,-0.014085259,-0.012723175,-0.030998945,-0.015136064,-0.0401375,-0.026463488,0.038578495,0.07998541,-0.014237099,-0.010529831,0.056128614,0.037907675,-0.059097726,0.05417056,0.082958736,-0.009134252,-0.054260757,-0.054202065,-0.11618847,-0.01405174,0.06535145,0.0035568997,0.066524446,-0.044019893,-0.052614007,-0.02574039,0.0647054,0.02392249,0.054227516,-0.015297123,0.0065662595,-0.03983744,-0.034733657,0.049171805,0.028220689,0.047715954,0.011627277,0.028319554,-0.0335994,-0.057167415,-0.007221433,0.053264305,0.040373173,0.047325287,-0.031557,-0.024102153,0.0097392,-0.01674235,0.0025580155,0.07028366,-0.0071781958,0.048924275,0.058978774,0.10782296,-0.025976786,0.05228242,-0.0807948,0.016916502,-0.007143717,0.013558098,0.073676944,0.06505334,0.008616979,-0.008804941,0.020318098,0.017507706,0.03264104,-0.030996464,-0.051108707,0.065104425,-0.045413326,0.011390516,-0.0033082915,0.0034082832,-0.024198843,-0.0013371801,-0.039690766,-0.009272256,-0.003407777,0.032495137,0.005419534,0.053965498,-0.065846704,-0.016936144,-0.006125944,-0.069586426,-0.022663658,0.08532246,0.03333747,-0.00494643,0.03444218,-0.08148192,-0.025534976,-0.021113764,-0.039273854,-0.03952054,-0.013458648,-0.03539913,-0.0833294,-0.015726525,0.0031500275,-0.054838143,-0.08316674,-0.008234053,-0.04711777,0.014021474,0.0038984169,-0.0016151736,0.10147354,0.010448409,0.00032858917,0.026777389,0.029944735,-0.06901488,-0.020588674,0.013908472,-0.023972617,-0.032528333,0.01995329,-0.062464606,-0.0032093867,-0.024792416,-0.019842254,0.02216227,0.10095912,-0.00013532673,-0.06238864,0.009707121,0.061033435,0.029572275,-0.025146998,-0.009804119,-0.0012242632,0.036190264,0.04167094,0.070770465,0.00019045723,0.070379324,-0.027354088,0.023488542,-0.04285553,0.04608597,-0.03795531,-0.020076795,-0.016664244,0.002103202,0.076279975,0.014466379,-0.05863308,-0.0011942231,0.031587236,-0.0052984487,-0.022511663,0.010679823,0.0037292014,-0.0491425,-0.050583497,-0.017251445,0.050162632,0.07143429,-0.059287854,0.062231027,0.06812368,-0.0008954576,0.011435777,-0.0053202068,0.031668555,0.016711306,-0.016464323,-0.008186157,-0.04822109,-0.07106247,-0.0063736793,0.030233445,0.022063958,0.045081083,0.0010316246,0.012626869,0.037735008,-0.028684152,-0.031325974,-0.058071256,-0.0077949753,0.0034812076,0.020687997,0.028490815,0.015027298,-0.047415506,0.017814463,0.03597448,0.01588718,0.035899296,-0.011932345,0.055948205,0.022845456,0.024084052,-0.068927325,-0.0392059,0.042272683,-0.035509475,0.0009178401,0.003042781,-0.09805096,0.0774122,-0.032059833,0.081539154,-0.103404194,0.049267806,0.0077401903,0.043843955,0.06927865,0.053415615,-0.021975603,-0.03251005,0.01496616,0.054724693,0.039631978,-0.030209128,0.03321391,-0.028865963,-0.009142891,-0.027479805,-0.11284759,-0.0081263855,0.03391427,0.020856364,-0.016265333,-0.06893102,0.041108705,-0.012176722,0.0100438325,0.03605663,0.018166753,-0.01800126,-0.0015882372,-0.00935341,-0.11777562,-0.02661155,0.049852714,0.05478877,0.0116448505,0.048344146,0.040289328,-0.059416283,0.07929439,-0.050501965,-0.07170737,-0.08357501,-0.07868208,-0.06089051,-0.023350397,0.041271232,0.032811668,0.06771606,0.037916698,-0.037558146,-0.040617976,-0.015999798,-0.06365382,-0.031791653,0.024503881,0.031561907,-0.04159506,0.07115487,0.054092735,0.04574948,-0.073632464,0.037537847,0.02533216,0.008346639,0.047948938,-0.02132105,-0.022232182,-0.06538253,0.108364865,0.056544162,0.0005101085]),
np.array([-0.07798266,-0.054283764,0.05052968,0.046649523,0.021070948,-0.03105914,0.040521782,0.056859095,0.007418541,-0.00809031,0.050815716,-0.018410787,-0.02509829,-0.09034279,0.07442135,-0.028322114,-0.0173977,-0.0521091,-0.006466795,-0.055036455,-0.015994377,-0.020602789,0.020658791,-0.051475998,0.09146341,-0.08325626,-0.07819381,-0.007998885,0.05964236,0.05747592,-0.061709885,0.04464765,-0.03550344,0.022440324,-0.0577231,-0.020224452,0.024534967,-0.0076288916,-0.048517194,-0.0179187,-0.0022572004,0.010363997,0.10056683,0.020754017,-0.06954198,0.015889645,-0.14211097,0.027642244,0.0429382,-0.008486895,-0.07953878,0.06558014,-0.071486056,0.03448279,0.020238023,0.054466188,-0.010280441,0.031519294,-0.053201783,-0.0075545735,0.031894516,0.040213693,-0.06466405,0.06035132,-0.014087551,0.0105966525,-0.017423153,-0.040713757,-0.09378713,-0.0053321505,-0.060052253,-0.015152287,0.049323544,-0.043319654,-0.020361915,-0.0029661744,-0.002309065,0.071649805,0.024616163,-0.05925753,-0.04454586,-0.015837086,-0.012223557,0.011779984,0.0456354,-0.03712523,0.010403782,-0.059639692,0.057458177,-0.030176034,0.030762099,0.03891118,-0.005941966,-0.074417815,-0.03269663,-0.049022514,0.024893884,0.06268444,-0.03881558,0.013626968,0.07609783,0.014174687,0.06976918,-0.005849624,0.050544765,0.044642713,-0.044252817,0.010935628,-0.07895465,-0.0019407271,0.016136816,-0.03379748,-0.055880748,-0.0028427874,-0.061316412,0.02629522,0.049009815,-0.032595605,-0.038793966,-0.0020276,0.021209324,-0.03264069,-0.032160107,0.042111684,-0.0007330165,0.0878859,-0.022615915,0.048561446,-0.05162,0.013580491,0.030583395,-0.021702457,-0.0127475085,0.032927066,0.10047982,-0.047430586,0.0573959,0.015046595,-0.04528808,0.0245643,-0.064784884,-0.07511515,0.038301885,-0.036501333,0.0073941555,0.001221284,0.06074201,-0.006703652,-0.08823157,0.018273935,-0.02509118,0.07009964,-0.07261378,-0.0027543227,0.0028729206,0.07089443,-0.050239407,0.050253455,0.041822657,-0.0008482563,-0.016990315,0.07190868,0.020620838,-0.040667064,-0.013867897,-0.019373678,0.065040566,0.09915693,-0.0738996,-0.032987874,0.058214955,0.041907225,-0.08024591,-0.010658122,-0.088508904,-0.045913465,-0.07937248,0.023383167,-0.059106745,-0.011505188,0.028212586,-0.065596364,-0.003102135,-0.0092996685,0.042424824,-0.021584917,-0.0023153485,0.01832521,0.04271218,-0.01739501,-0.009656324,0.01324175,-0.012530245,-0.051606964,-0.03509125,0.0147413565,-0.043380037,0.04707074,0.013150907,0.046448156,-0.008789048,0.06734979,-0.050221972,0.031800587,-0.02363081,0.009649102,-0.04820691,-0.0022902158,-0.033537965,-0.018312749,0.023790741,0.009696334,0.0659785,0.040766776,0.018288486,-0.02690822,-0.028096966,-0.020275537,0.0174787,-0.026718771,-0.018565547,0.013353647,0.03924779,0.072877206,-0.022040732,-0.033518337,0.017213684,0.007912718,0.0152721405,-0.035093207,0.07267436,-0.031054989,-0.061815463,0.034950897,0.0062858425,-0.0145139,-0.046769973,0.0042025535,-0.04109427,-0.020761393,0.014807373,-0.01136967,0.051902343,-0.012116286,0.01690475,0.048997592,0.08793374,-0.038198717,-0.0013871583,0.051895924,-0.038114604,0.006659311,-0.023632629,-0.11725031,-0.036154673,0.008517405,0.00997228,0.044096034,-0.05751039,-0.031647943,0.0029028445,0.017047714,0.006255547,0.058554117,-0.0029046088,-0.020351727,-0.06963696,0.000024455841,0.0671056,0.0151265245,0.00848875,0.0039519574,0.0066434685,-0.051079314,-0.045967773,0.019034138,0.09284751,0.012712466,0.05556509,0.012692569,-0.024493964,0.0013555345,-0.0099218935,0.0060297977,0.05356134,-0.008041604,0.029324252,0.10564803,0.085755624,-0.024246717,0.042220484,-0.07390136,0.0065074274,-0.02468345,-0.008568837,0.06769956,0.059352595,0.008873183,-0.032315996,0.045309,0.0155057805,0.05393485,-0.03856007,-0.02467162,-0.004373017,-0.02560942,0.010127752,-0.011140363,-0.0153578995,-0.022791477,0.012104956,-0.045441106,-0.025558878,-0.008336808,0.09528999,0.003923222,0.06778922,-0.046550475,-0.058125827,0.018858073,-0.0062780073,-0.04001839,0.055446357,-0.0023149652,0.070320874,0.034068156,-0.022123326,-0.004252216,-0.015499629,-0.015522463,-0.04376197,0.00046108937,-0.023433927,-0.06850954,0.0034564144,0.029163236,-0.039824206,-0.083219096,0.026682716,-0.053873193,0.01761119,0.009945783,-0.0169687,0.068088934,-0.0040775794,-0.025579063,0.046371795,-0.02959913,-0.0036507244,-0.03106475,-0.010327854,-0.0125609245,-0.016775241,-0.008970706,-0.091764316,0.024132906,-0.017156705,-0.05256401,-0.016718654,0.071802996,-0.02214211,-0.05534577,-0.00092294696,0.062417872,0.060629945,-0.06111194,-0.0060453773,-0.037070468,0.04007508,0.034695208,0.06765697,0.03403058,0.0072104894,-0.055666003,0.072291575,-0.055956304,0.078700446,-0.03148207,-0.0047910316,-0.016794477,0.031048337,0.047248293,0.022077838,-0.092380606,-0.025556834,0.001646558,0.005172259,0.03992773,0.046799164,0.018080616,-0.036914002,-0.007863383,-0.0363367,0.014806555,0.072577395,-0.08061511,0.04485788,0.07864235,-0.017305404,0.04473359,0.007111581,0.023531746,0.011994637,-0.03600955,-0.021302734,-0.030761585,-0.034397732,-0.0038077014,0.01591895,0.035187613,0.076990575,0.015482335,-0.013829828,0.003115747,-0.02467947,-0.0711409,-0.08963496,-0.025385398,-0.009250614,-0.029377762,0.058076374,0.0116989715,-0.013783686,0.04195252,0.026131814,0.024278319,0.06452872,0.01662188,0.094086215,0.027064191,-0.008453934,-0.06914073,-0.016542032,0.03447861,-0.031123942,0.039868843,-0.02094263,-0.08964251,0.052045636,-0.050372995,0.10945851,-0.06785519,0.02672192,0.03568874,0.034485642,0.081689544,0.029056864,-0.02175776,-0.05406794,0.031915367,0.04831225,0.04945616,-0.038729835,0.062793925,-0.004316105,-0.017714294,-0.0048810146,-0.06156778,0.037696395,0.01036689,0.016157381,-0.0018594797,-0.061274663,0.034879386,0.009632864,-0.04157344,0.057582498,0.007323124,0.033146225,-0.011602822,-0.028879212,-0.09098352,-0.009962591,0.03859099,0.07082804,0.021792335,0.02989589,0.04104887,-0.03484907,0.060376167,-0.065522924,-0.05521059,-0.036732998,-0.0922379,-0.06927129,-0.003897457,0.012586928,0.016975619,0.043423608,0.017030505,-0.076973565,-0.04766958,-0.010868287,-0.031062126,-0.07254663,0.02245004,0.047176823,-0.02107026,0.041862234,0.03629617,0.050140303,-0.04328105,0.04268356,-0.00744794,-0.0437865,0.014638516,-0.009749915,-0.051186956,-0.10666277,0.07455595,0.043210186,-0.010982545]),
np.array([-0.07196909,-0.05147855,0.022652945,0.05067851,-0.012647839,-0.016674947,0.0148819005,0.055728532,-0.015315111,-0.03265465,0.035869706,-0.018993355,-0.038135346,-0.09298946,0.07199422,-0.027881611,-0.016768893,-0.010594383,-0.048293907,-0.041076303,-0.03099977,-0.024772339,0.044483595,-0.06064177,0.08018772,-0.03651016,-0.05553724,0.0060675708,0.053755082,0.05145266,-0.045172863,0.032665994,0.032756746,-0.030155893,-0.061887085,0.0090222545,0.04723522,0.03726469,-0.054588847,0.005242312,-0.005440875,-0.023302529,0.1164574,0.059321355,-0.04210087,-0.006497513,-0.085154034,0.022971421,0.060666382,0.008642739,-0.049994346,0.050760377,-0.06464577,0.035288297,0.034184385,0.044319168,-0.026078677,0.034070864,-0.034948032,0.011656565,0.06131566,-0.008948989,-0.02579597,0.0492967,-0.037932996,-0.01256198,-0.031457033,-0.028484382,-0.09719903,0.033999577,-0.070180535,0.02101209,0.05160254,-0.026859306,-0.014951534,-0.024508892,0.0013883515,0.10908598,0.011718336,-0.08825903,-0.11146539,-0.030563824,-0.02483478,0.00806371,0.007311823,-0.04280394,0.008059742,-0.067954324,0.040424317,-0.024708582,0.020533677,0.04484581,0.0038887768,-0.11844208,-0.035764843,-0.039053295,0.05653851,0.094268605,0.002479965,0.056893084,0.05243562,0.019115482,0.08178443,-0.028012872,0.035073087,0.01738713,-0.05232203,-0.020549476,-0.029016646,0.018164895,0.008498571,-0.029640852,-0.0396236,-0.015239622,-0.040092107,0.029149186,0.07142547,-0.030418264,0.011038961,-0.024302527,0.057097837,-0.06514985,-0.03936791,0.04931567,0.019205054,0.039611425,-0.06724317,0.020350996,-0.037397884,0.022786569,-0.052229464,-0.013916021,0.0049099512,0.048014328,0.04879093,-0.059065446,0.03410038,-0.019146744,-0.033900265,0.08160459,-0.030579654,-0.038723905,0.041636117,-0.029401774,0.0039169877,0.02700207,0.04612366,-0.04128822,-0.056177586,0.023443218,0.0135029,0.03240849,-0.09216204,-0.024405355,-0.0059117973,0.039817538,-0.049721926,0.00479006,0.02122715,-0.01581795,-0.053789236,0.03901376,0.026133899,-0.04991461,-0.038344454,-0.05373591,0.08392013,0.12498886,-0.053144634,-0.0059957397,0.03388029,0.040412568,-0.10110563,-0.041511558,-0.07513612,-0.038963545,-0.038814876,0.022356205,-0.09468192,-0.021025294,0.018690335,-0.06856256,-0.02876724,-0.03851056,-0.018069081,-0.029945014,-0.035569325,-0.00651869,0.035394687,-0.035366166,-0.033798415,0.0011862242,0.013220759,-0.06716318,-0.02583013,-0.027922383,-0.048614785,0.035554536,-0.0048779687,0.021026842,-0.010994067,0.0657252,-0.0568282,-0.0072939657,-0.0453639,0.022777116,-0.019500932,-0.027309034,-0.04523009,0.023484807,0.06451281,0.0042394064,0.066909544,0.0003314272,-0.00043410994,0.008314098,-0.030244974,-0.01800424,-0.0064961864,-0.0136998715,0.0019894186,-0.006723449,-0.021313353,0.033883393,-0.010478537,-0.07144829,0.028799798,0.005597329,-0.024480766,-0.015373557,0.04641585,-0.04792437,-0.0527407,0.05552841,0.0146437995,-0.037181906,-0.022472361,0.04158607,-0.023041073,-0.007889361,0.003938118,-0.000524121,0.07177027,-0.006605247,0.005361352,0.04618099,0.06146947,-0.021937715,0.041772515,0.046824932,-0.046210457,-0.0022079444,-0.028235164,-0.07539568,-0.014659098,0.00521537,0.031906612,0.061672404,-0.053870603,-0.021238072,-0.0016417527,0.047237936,-0.029083792,0.06963059,0.036478505,-0.00041907566,-0.07749392,-0.007011339,0.053633712,-0.0055037495,-0.038425148,0.013585909,0.031190928,-0.06865299,-0.057710797,0.028537717,0.04526516,0.020024752,0.041543003,0.0033079744,-0.046707153,0.065502234,0.042611856,0.0017289099,0.039335586,0.017715491,0.004073125,0.057086937,0.034859527,-0.028219454,0.036441825,-0.089852415,-0.021570237,-0.018363494,-0.012586351,0.048043065,0.011450159,0.00072140264,-0.017732508,0.005329022,0.024764583,0.09753075,-0.052937314,-0.0283945,0.04165624,0.005299832,0.061719246,-0.038112868,-0.02750109,-0.03370762,-0.0014716461,-0.017873509,-0.024024105,-0.03257181,0.09695087,0.01249321,0.10020778,-0.04025076,-0.060647342,0.017317735,0.037390627,-0.042384766,0.058908932,0.017202394,0.032762635,0.049689073,-0.022053566,-0.05287697,-0.014187679,0.008363001,-0.02459162,0.0013951323,-0.024205027,-0.060468797,0.0003692801,-0.010470724,-0.005525802,-0.08174348,-0.0019858493,-0.07912048,0.030802911,0.035113543,-0.008354592,0.08381294,0.03080345,0.00023582777,0.042839367,-0.05484613,-0.035685364,-0.011340793,-0.025292542,-0.01032753,-0.03729605,0.037445717,-0.07516498,-0.006181009,0.0051571345,-0.04897975,-0.026642155,0.12578765,-0.028862562,-0.07275555,0.011456582,0.08161533,0.015573906,-0.03531863,0.00033946373,-0.0136873545,0.03415442,0.06446963,0.052853372,0.029883534,0.064058125,0.0031778621,0.022592848,-0.050506763,0.057378832,-0.051747393,-0.01798953,-0.018835053,-0.030058485,0.013227344,0.027007181,-0.12660724,-0.001966039,-0.0124565,0.018159168,0.07829186,0.039425213,0.024493493,-0.037621737,-0.06021711,-0.023681184,0.020888705,0.07578398,-0.052895032,0.007436754,0.08077827,0.015981479,0.022777764,0.023603145,0.024559326,-0.00024400801,-0.008566272,-0.03540397,-0.03511101,-0.06434624,-0.037165716,0.011255511,0.019190209,0.09728656,0.04225528,-0.022339802,0.04337476,0.0073011015,-0.032359067,-0.048336703,-0.05697014,0.0068408647,-0.027747627,0.034022845,0.026456838,0.007814532,-0.008401661,0.034905635,0.04000224,0.04892535,0.014965273,0.023323998,0.008985537,0.00999829,-0.039920654,-0.016956264,0.035249688,-0.048277684,0.03746751,0.01476723,-0.07167306,0.044094186,-0.03009398,0.10972194,-0.0600444,0.016823333,0.01152613,0.067495935,0.06836543,-0.0027340285,-0.037945505,-0.03569149,0.0147977965,0.029825624,0.043721363,-0.004932114,0.06419617,-0.021271199,0.022942806,-0.017601747,-0.08092604,0.03636101,-0.009441738,0.03682031,-0.00395771,-0.08299885,0.06572029,-0.009083251,-0.03633785,0.058966573,-0.007015063,0.07542244,0.020332526,-0.027168952,-0.090319745,0.0032182967,0.07188774,0.08899028,0.030589718,0.039473794,0.034792725,-0.031093441,0.018665532,-0.030979438,-0.02370818,-0.06481518,-0.082514025,-0.043442044,-0.0059411232,-0.03925425,0.020910379,0.034160163,0.020642469,-0.08026184,0.00019704732,-0.0057644607,-0.028994743,-0.0020527972,0.03175117,0.052597765,-0.05871802,0.06959176,0.052835494,0.04572727,-0.03226354,0.06892118,0.0108810235,0.023045206,0.06365664,-0.024808409,-0.00010990917,-0.07928286,0.045387242,0.094515465,-0.0015757083]),
np.array([-0.08384529,-0.056995634,0.00955793,0.0714702,-0.0070659723,-0.024235409,0.018402677,0.06556553,-0.0061800657,-0.0031979724,0.025841147,0.0054110903,-0.022494646,-0.082519405,0.055053927,-0.027768172,-0.024252871,-0.0023564165,-0.048758157,-0.07060738,-0.028761355,-0.048223775,0.042887446,-0.006373214,0.06360489,-0.026559142,-0.04280241,0.05354887,0.0940171,0.056343205,-0.051226433,0.058902107,-0.031090407,0.016858995,-0.07118802,0.03863912,0.026864737,0.007993523,-0.06454065,-0.016604327,-0.035971634,-0.00027499997,0.12546684,0.007866036,-0.07656036,0.016988026,-0.08729815,0.03266644,0.039291188,0.022963857,-0.07726852,0.05021976,-0.06649904,0.018559549,0.0040197656,0.04094515,-0.02262635,0.034208674,-0.016348595,0.009193605,0.057319924,-0.017346853,-0.037342355,0.08074207,-0.020169325,-0.090270646,-0.05844592,-0.021655928,-0.065521814,0.037557118,-0.06634537,0.015446023,0.07851297,-0.06301101,0.033363875,0.0032729942,-0.009123367,0.08008837,0.005800538,-0.068395406,-0.074375115,-0.024219941,-0.022517245,0.041581642,0.02377621,-0.024362082,0.0034839918,-0.005810964,0.0613876,-0.027784513,-0.009169569,0.019763647,0.024417726,-0.06376603,0.005321417,-0.03775039,0.06708585,0.038808983,-0.0031903142,0.033691272,0.09519017,0.032584872,0.09078567,0.01668372,0.00680209,0.068776526,-0.08940596,-0.02059819,-0.06718856,-0.059758138,-0.013206891,0.00795151,-0.010195872,0.016442942,-0.03560236,0.031180006,-0.006599329,-0.045772947,-0.023177098,-0.029496456,0.07176304,0.0023112902,-0.006745487,0.04539381,0.028933281,0.074462056,-0.022978764,0.0029137759,-0.06789499,0.0090543,-0.040764943,0.009222888,-0.028711163,0.022283586,0.06242839,-0.04861349,0.039073963,-0.0064405585,-0.025544947,0.048006583,-0.06406216,-0.030073233,0.025133388,-0.0792998,-0.038337838,-0.013220894,0.025908062,-0.026799656,-0.093650624,-0.0003914961,0.016028997,0.008046647,-0.09279257,-0.012240875,-0.0005200626,0.073296756,-0.042748965,0.051294785,0.05272209,-0.0043234536,0.0036478543,0.03306709,0.00910223,-0.06660361,-0.0040395083,-0.0150434915,0.077919446,0.12266638,-0.08247044,-0.012319436,0.09070857,0.0089000985,-0.098627344,-0.032820687,-0.07909937,-0.028773507,-0.039072525,0.036201622,-0.009450375,-0.013459493,0.018048545,-0.037687782,-0.030955175,0.0105400765,0.038163595,-0.026198072,0.012176599,-0.015521553,0.030752474,0.006347629,-0.021630537,0.023346411,0.029520376,-0.046465173,-0.016328115,0.039492026,0.010108575,0.052507676,-0.040637113,0.085432395,0.03159048,0.044994514,-0.03278362,0.007250705,-0.04388728,0.02595286,-0.023788346,-0.03927426,-0.017410258,0.03815414,0.06422528,0.06290276,0.049886834,0.0011184837,-0.0032311575,-0.02640324,0.010222874,-0.0627848,0.014589991,-0.014694302,0.00011193397,0.034846477,-0.028288228,0.022192687,0.0027343123,-0.06036578,0.019734701,0.045406282,-0.018194746,-0.07185966,0.025574422,-0.043592658,-0.078854926,0.011959127,0.016054885,-0.014884717,-0.03678632,0.028956562,-0.011051496,-0.03407644,-0.04936756,0.050570864,0.056030847,-0.013272257,-0.026951924,0.087265275,0.07914401,-0.026169272,0.023077894,0.035158534,-0.031813495,-0.007892901,-0.04305087,-0.079066455,0.00265578,0.066151984,0.029088957,0.0551881,-0.08005578,-0.03161518,0.010223464,0.043731753,-0.0344324,0.06626603,0.013790746,-0.036468178,-0.039795335,-0.0032293664,0.064336255,-0.027297948,0.0034952767,0.020599207,0.03990188,-0.088165745,-0.07929271,0.0077135456,0.06338514,0.02569586,0.039485853,-0.006859275,-0.06053028,-0.042636126,-0.029884288,0.00792015,0.029888628,-0.005305352,0.051011596,0.06183487,0.06413854,0.008890989,0.010411711,-0.059827544,0.0073146876,0.0015531096,-0.04957749,0.03341601,0.042921994,0.030360606,-0.040001724,0.018268099,0.04490135,0.06537951,-0.04869247,-0.0035582886,0.047803108,-0.00536961,0.029795922,0.0068575456,-0.013830336,-0.043887775,0.01956496,-0.035108536,-0.01765747,-0.03393048,0.06292075,0.02902973,0.061588723,-0.017476793,-0.021278001,-0.013319522,-0.081841975,-0.009697805,0.032978065,0.051296465,0.039187223,0.034011826,-0.03629721,-0.027286837,-0.040861536,-0.046344373,-0.066226415,-0.012209852,-0.019896816,-0.080798596,0.027092222,-0.007871218,-0.037442982,-0.06894513,-0.006391965,-0.047458,0.03298129,0.03814616,-0.022375626,0.051220354,0.004873042,0.014818746,0.012056001,-0.025325878,-0.032999285,-0.034216598,-0.0227554,-0.04740638,-0.061359745,0.017876187,-0.090746224,-0.019620415,-0.021115532,-0.08250128,-0.0256937,0.12619194,-0.001073395,-0.06567903,0.0049925437,0.085715875,0.007988665,-0.047876798,-0.0489514,0.018101465,0.062627174,0.027069418,0.06826171,-0.00028074742,-0.0075205574,-0.021937808,0.034143638,-0.0061169127,0.06720808,-0.003920912,0.006751027,-0.034782786,-0.013148503,0.032599114,0.021856572,-0.04990026,-0.017199473,0.020663338,0.025836596,0.029579751,0.032709114,0.0094216755,-0.043117236,-0.049878962,0.016973447,0.0113147115,0.10190329,-0.058143336,-0.0045849453,0.06341958,-0.030067058,0.027860876,-0.02301842,0.050017174,-0.013672709,0.007880632,-0.039798234,-0.00402037,-0.051138006,0.00925456,0.02163706,-0.0074591655,0.08427085,0.00879441,0.03110402,0.023207698,0.0028510499,-0.028778719,-0.04101072,-0.034779917,-0.0052601094,0.0091732945,0.0411731,0.038964912,-0.0129775135,0.017948551,0.032196105,0.0026129102,0.013302286,0.0012186564,0.04674883,0.03037444,-0.010827984,-0.044464815,-0.062791295,0.016156908,-0.056369234,0.02428967,0.021431364,-0.1095657,0.054888565,0.005611504,0.107445545,-0.07912847,-0.004103787,0.03753277,0.025123745,0.07676512,0.031806573,-0.014637481,-0.0027460596,0.05053382,0.08194496,0.01074578,0.0047812033,0.046548042,-0.035667,0.021582818,-0.0029361453,-0.08240759,0.03729628,0.031600427,0.03239909,-0.014937541,-0.031444304,0.017337516,-0.0069175973,-0.0010631362,0.050010264,-0.010508765,0.031440075,0.029943692,-0.040960878,-0.11539751,-0.02434986,0.07442608,0.054929715,0.0073242136,0.054951474,0.010772357,-0.06728155,0.08500287,-0.038237732,0.013910674,-0.053832978,-0.11429846,-0.08187077,0.0030970597,-0.032125972,0.013817181,0.012862175,0.038733684,-0.015591849,-0.04053012,-0.0014433007,-0.048451178,-0.023375222,0.060041893,0.04465183,-0.05434533,0.05260507,0.012489562,0.044327267,-0.053976137,0.02196005,0.028759168,0.0013878447,0.04464863,-0.03263315,-0.04273713,-0.08373826,0.04386464,0.034007005,-0.010802366]),
])

In [95]:
similarity = doc_embeddings_v2.dot(question_vector)
similarity

# Is it different from Q3? If yes, why?
# It is indeed different from the result on Q3, but qe still have the first document with the best similarity.
# The results changed because now we have embedded the answer to the question together with the question itself, thus changing the semantical value of each document.

array([0.85145433, 0.84365942, 0.84082871, 0.7755158 , 0.80860079])

In [96]:
# Selecting the embedding model
# min dimensionality in fastembed: 384
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [97]:
EMBEDDING_DIMENSIONALITY=384
model_handle = "BAAI/bge-small-en"

In [98]:
#Indexing with qdrant
collection_name = 'indexing_with_wdrant'

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  8.56it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
query_points = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=question,
        model=model_handle 
    ),
    query_filter=models.Filter( 
        must=[
            models.FieldCondition(
                key="course",
                match=models.MatchValue(value=course)
            )
        ]
    ),
    limit=5,
    with_payload=True
)